# Phan dual

Câu 2. (4 điểm) Cho ma trận $A=\begin{bmatrix}1 & 0 & 0\\ 0 & 2 & 0 \\ 0 & 0 & 3 \end{bmatrix}$ và vector $b=(1/2;1; 3/2)$. Xét bài toán tối ưu sau $\begin{align}
\min_{x=(x_1,x_2,x_3)\in\mathbb{R}^3}\;\;\; &f_0(x)=\sum_{i=1}^3x_i\log x_i\\
\text{subject to }\;\;\; & Ax\geq b, \quad\quad\quad\quad \quad (1) \\
& \mathbf{1}^Tx=\dfrac{3}{2}.
\end{align}$
a) (1 điểm) Xác định hàm Lagrange $L(x,\lambda,v)$ cho bài toán $(1)$.

b) (2 điểm) Gọi $x^*$ và $(\lambda^*, v^*)$ lần lượt là điểm tối ưu của bài toán $(1)$ và bài toán đối ngẫu của nó. Viết điều kiện Karush-Kuhn-Tucker (KKT) cho $x^*$ và $(\lambda^*, v^*)$.

c) (1 điểm) Tìm một giá trị của $x^*$ và $(\lambda^*, v^*)$ thoả điều kiện KKT vừa tìm được ở câu b), từ đó đưa ra giá trị tối ưu cho bài toán $(1)$.

Lưu ý: sinh viên hoàn thành Câu 2 theo mẫu dưới đây.

a) Hàm Lagrange $L(x,\lambda,v)$

 
a) Hàm Lagrange $L(x,\lambda,v) = f_0(x) + \lambda^T(Ax - b) + v(\mathbf{1}^Tx - \frac{3}{2})$.



In [ ]:
import numpy as np
from sympy import symbols, Matrix, log, diff, simplify

# ===============================================================
# 🎯 a) Hàm Lagrange
# ===============================================================
def lagrangian_function():
    """
    Trả về biểu thức L(x, λ, v) của bài toán.
    """
    # Khai báo biến
    x1, x2, x3, l1, l2, l3, v = symbols('x1 x2 x3 l1 l2 l3 v', real=True, positive=True)

    # Ma trận A và vector b
    A = Matrix([[1, 0, 0],
                [0, 2, 0],
                [0, 0, 3]])
    b = Matrix([1/2, 1, 3/2])
    x = Matrix([x1, x2, x3])
    lam = Matrix([l1, l2, l3])

    # Hàm mục tiêu
    f0 = sum(x[i] * log(x[i]) for i in range(3))

    # Hàm Lagrange
    L = f0 + lam.dot(A*x - b) + v*(sum(x) - 3/2)

    return simplify(L), (x, lam, v)


# ===============================================================
# 🎯 b) Viết điều kiện KKT
# ===============================================================
def kkt_conditions():
    """
    Trả về hệ phương trình KKT ở dạng symbolic.
    """
    L, (x, lam, v) = lagrangian_function()
    x1, x2, x3 = x
    l1, l2, l3 = lam
    v = v

    # Gradient theo x
    grad_x = [diff(L, xi) for xi in x]

    # Điều kiện KKT:
    # 1. ∇x L = 0
    # 2. λ_i (a_i^T x - b_i) = 0
    # 3. λ ≥ 0
    # 4. a_i^T x - b_i ≥ 0
    # 5. 1^T x = 3/2
    A = np.array([[1, 0, 0],
                  [0, 2, 0],
                  [0, 0, 3]], dtype=float)
    b = np.array([0.5, 1.0, 1.5])

    return grad_x, (A, b)


# ===============================================================
# 🎯 c) Tìm nghiệm thỏa KKT
# ===============================================================
def solve_kkt():
    """
    Giải gần đúng hệ KKT bằng Newton-Raphson.
    Do f(x)=∑x log x là strictly convex, nghiệm duy nhất tồn tại.
    """
    # Dữ liệu
    A = np.array([[1, 0, 0],
                  [0, 2, 0],
                  [0, 0, 3]], dtype=float)
    b = np.array([0.5, 1.0, 1.5])
    ones = np.ones(3)

    # Giả sử ràng buộc Ax = b hoạt động tại biên (do dạng log barrier)
    # => x_i = b_i / a_ii
    x = np.array([0.5, 0.5, 0.5])  # điểm khởi tạo
    v = 0.1                        # hệ số ràng buộc equality
    lam = np.ones(3)               # khởi tạo λ

    for _ in range(20):
        # Gradient của L theo x: ∇f0(x) + A^T λ + v*1 = 0
        grad = np.log(x) + 1 + A.T @ lam + v*ones

        # Điều kiện Ax - b ≥ 0
        feas = A @ x - b

        # Cập nhật sơ bộ (Newton step giảm dần)
        x = np.maximum(x - 0.1 * grad, 1e-6)
        lam = np.maximum(lam - 0.1 * feas, 0)
        v = v - 0.01 * (ones @ x - 1.5)

        if np.linalg.norm(grad) < 1e-6:
            break

    f_opt = np.sum(x * np.log(x))
    return x, lam, v, f_opt


# ===============================================================
# 🎯 In kết quả
# ===============================================================
if __name__ == "__main__":
    # a) Lagrange function
    L, _ = lagrangian_function()
    print("a) Hàm Lagrange:")
    print("L(x, λ, v) =", L)

    # b) Điều kiện KKT
    grad_x, (A, b) = kkt_conditions()
    print("\n\nb) Điều kiện KKT:")
    print("∇x L =", grad_x)
    print("λ ≥ 0,  Ax - b ≥ 0,  λᵢ(Ax - b)ᵢ = 0,  1ᵀx = 3/2")

    # c) Giải gần đúng
    x_star, lam_star, v_star, f_opt = solve_kkt()
    print("\n\nc) Nghiệm gần đúng thỏa KKT:")
    print(f"x* = {x_star}")
    print(f"λ* = {lam_star}")
    print(f"v* = {v_star:.4f}")
    print(f"Giá trị tối ưu f(x*) = {f_opt:.6f}")


In [ ]:
import sympy as sp
import numpy as np


# ============================================================
# HÀM XÂY DỰNG & GIẢI BÀI TOÁN KKT CHUNG
# ============================================================
def solve_entropy_KKT(A, b, c):

    A = np.array(A, float)
    b = np.array(b, float)
    m, n = A.shape

    # -----------------------------
    # Tạo biến x
    # -----------------------------
    x_symbols = sp.symbols(f"x1:{n+1}", real=True, positive=True)
    x = sp.Matrix(x_symbols)

    # -----------------------------
    # Tạo biến dual λ ≥ 0 và v tự do
    # -----------------------------
    lam = sp.symbols(f"l1:{m+1}", real=True, nonnegative=True)
    lam_vec = sp.Matrix(lam)

    v = sp.Symbol("v", real=True)

    # -----------------------------
    # Hàm mục tiêu f(x)
    # f = sum x_i log x_i
    # -----------------------------
    f_expr = sum([x_symbols[i] * sp.log(x_symbols[i]) for i in range(n)])

    # -----------------------------
    # Lagrangian
    # -----------------------------
    Ax_minus_b = sp.Matrix(A) * x - sp.Matrix(b)

    L = (
        f_expr
        + lam_vec.dot(Ax_minus_b)
        + v * (sum(x_symbols) - c)
    )

    print("\n========== LAGRANGIAN ==========")
    sp.pprint(L)

    # -----------------------------
    # Stationarity ∂L/∂x = 0
    # -----------------------------
    grad_L = [sp.simplify(sp.diff(L, x_symbols[i])) for i in range(n)]

    print("\n========== STATIONARITY CONDITIONS ==========")
    for i, g in enumerate(grad_L):
        print(f"dL/dx{i+1} = {g} = 0")

    # -----------------------------
    # Primal feasibility
    # -----------------------------
    print("\n========== PRIMAL FEASIBILITY ==========")
    for i in range(m):
        print(f"A[{i}]x - b[{i}] = {sp.simplify(Ax_minus_b[i])} ≥ 0")

    print(f"sum(x_i) = {c}")

    # -----------------------------
    # Dual feasibility
    # -----------------------------
    print("\n========== DUAL FEASIBILITY ==========")
    for j in range(m):
        print(f"λ{j+1} ≥ 0")

    # -----------------------------
    # Complementary slackness
    # -----------------------------
    print("\n========== COMPLEMENTARY SLACKNESS ==========")
    for j in range(m):
        print(f"λ{j+1} * ({Ax_minus_b[j]}) = 0")

    # =====================================================
    # GIẢI KKT
    # =====================================================

    # build system:
    eqs = []

    # stationarity
    eqs.extend(grad_L)

    # equality constraint
    eqs.append(sum(x_symbols) - c)

    # complementary slackness
    for i in range(m):
        eqs.append(lam[i] * Ax_minus_b[i])

    unknowns = list(x_symbols) + list(lam) + [v]

    sol = sp.solve(eqs, unknowns, dict=True, simplify=True)

    if not sol:
        print("⚠ No symbolic KKT solution found")
        return None

    sol = sol[0]

    print("\n========== SOLUTION (x*, λ*, v*) ==========")
    for var in unknowns:
        print(f"{var} = {sp.simplify(sol[var])}")

    # compute objective
    x_star = np.array([float(sol[xx]) for xx in x_symbols])
    f_star = float(sum(x_star[i] * np.log(x_star[i]) for i in range(n)))

    print("\n========== OPTIMAL VALUES ==========")
    print("x* =", x_star)
    print("f(x*) =", f_star)

    return sol, x_star, f_star
A = [
    [1,0,0],
    [0,2,0],
    [0,0,3]
]

b = [0.5, 1, 1.5]
c = 1.5

solve_entropy_KKT(A, b, c)


In [ ]:
def solve_entropy_KKT(A, b, c, sense=">="):
    """
    sense: ">=" nếu ràng buộc gốc là Ax >= b
           "<=" nếu ràng buộc gốc là Ax <= b
    """
    A = np.array(A, float)
    b = np.array(b, float)
    m, n = A.shape

    x_symbols = sp.symbols(f"x1:{n+1}", real=True, positive=True)
    x = sp.Matrix(x_symbols)
    lam = sp.symbols(f"l1:{m+1}", real=True, nonnegative=True)
    lam_vec = sp.Matrix(lam)
    v = sp.Symbol("v", real=True)

    # f(x) = sum x_i log x_i
    f_expr = sum([x_symbols[i] * sp.log(x_symbols[i]) for i in range(n)])

    A_sym = sp.Matrix(A)
    b_sym = sp.Matrix(b)

    # ----- CHUẨN HÓA g(x) <= 0 -----
    if sense == ">=":
        # Ax >= b  <=>  b - Ax <= 0
        g = b_sym - A_sym * x
    elif sense == "<=":
        # Ax <= b  <=>  Ax - b <= 0
        g = A_sym * x - b_sym
    else:
        raise ValueError("sense must be '>=' or '<=' ")

    # Lagrangian
    L = f_expr + lam_vec.dot(g) + v * (sum(x_symbols) - c)

    print("\n===== LAGRANGIAN =====")
    sp.pprint(L)

    # Stationarity
    grad_L = [sp.simplify(sp.diff(L, xi)) for xi in x_symbols]
    print("\n===== STATIONARITY =====")
    for i, gi in enumerate(grad_L):
        print(f"dL/dx{i+1} = {gi} = 0")

    # Primal feasibility
    print("\n===== PRIMAL FEASIBILITY =====")
    if sense == ">=":
        for i in range(m):
            print(f"b[{i}] - a[{i}]^T x = {sp.simplify(g[i])} <= 0  (tức Ax >= b)")
    else:
        for i in range(m):
            print(f"a[{i}]^T x - b[{i}] = {sp.simplify(g[i])} <= 0  (tức Ax <= b)")
    print(f"sum(x_i) = {c}")

    # Dual feasibility
    print("\n===== DUAL FEASIBILITY =====")
    for j in range(m):
        print(f"λ{j+1} >= 0")

    # Complementary slackness
    print("\n===== COMPLEMENTARY SLACKNESS =====")
    for j in range(m):
        print(f"λ{j+1} * ({sp.simplify(g[j])}) = 0")

    # ===== GIẢI KKT =====
    eqs = []
    eqs.extend(grad_L)
    eqs.append(sum(x_symbols) - c)
    for j in range(m):
        eqs.append(lam[j] * g[j])

    unknowns = list(x_symbols) + list(lam) + [v]
    sol = sp.solve(eqs, unknowns, dict=True, simplify=True)

    if not sol:
        print("⚠ No solution found")
        return None

    sol = sol[0]
    print("\n===== SOLUTION =====")
    for var in unknowns:
        print(f"{var} = {sp.simplify(sol[var])}")

    x_star = np.array([float(sol[xi]) for xi in x_symbols])
    f_star = float(sum(x_star[i] * np.log(x_star[i]) for i in range(n)))
    print("\n===== OPTIMAL VALUE =====")
    print("x* =", x_star)
    print("f(x*) =", f_star)

    return sol, x_star, f_star
A = [
    [1,0,0],
    [0,2,0],
    [0,0,3]
]
b = [0.5, 1.0, 1.5]
c = 1.5

solve_entropy_KKT(A, b, c, sense=">=")


In [ ]:
import sympy as sp
import numpy as np


def solve_entropy_KKT(A_ineq=None, b_ineq=None, A_eq=None, b_eq=None):
    """
    Giải bài toán:
        minimize   f(x) = sum_i x_i log x_i   (entropy-like)
        subject to A_ineq x <= b_ineq   (bất đẳng thức)
                   A_eq   x = b_eq      (đẳng thức)

    - Tự sinh Lagrangian L(x, λ, v)
    - In điều kiện KKT (stationarity, feasibility, dual feasibility, slackness)
    - Tìm 1 nghiệm KKT (nếu Sympy giải được)
    - Trả về (sol_dict, x_star, f_star)

    Cách dùng cho các dạng:
    -----------------------------------
    1) Ax >= b      → truyền A_ineq = -A, b_ineq = -b
    2) Ax <= b      → truyền A_ineq =  A, b_ineq =  b
    3) Cx  = d      → truyền A_eq   =  C, b_eq   =  d
    4) Kết hợp cả bất đẳng thức + đẳng thức → truyền cả 2
    """

    # ---------------------------
    # Chuẩn hóa ma trận / vector
    # ---------------------------
    A_ineq_mat = None
    A_eq_mat   = None

    if A_ineq is not None:
        A_ineq_mat = np.atleast_2d(np.array(A_ineq, dtype=float))
        b_ineq_vec = np.array(b_ineq, dtype=float).reshape(-1)
        m_ineq, n_ineq = A_ineq_mat.shape
    else:
        m_ineq = 0

    if A_eq is not None:
        A_eq_mat = np.atleast_2d(np.array(A_eq, dtype=float))
        b_eq_vec = np.array(b_eq, dtype=float).reshape(-1)
        m_eq, n_eq = A_eq_mat.shape
    else:
        m_eq = 0

    # Xác định số chiều n
    n = None
    if A_ineq_mat is not None:
        n = n_ineq
    if A_eq_mat is not None:
        if n is None:
            n = n_eq
        else:
            if n != n_eq:
                raise ValueError("Số chiều của A_ineq và A_eq không khớp")

    if n is None:
        raise ValueError("Phải cung cấp ít nhất A_ineq hoặc A_eq để suy ra số chiều n")

    # ---------------------------
    # Tạo biến x
    # ---------------------------
    x_symbols = sp.symbols(f"x1:{n+1}", real=True, positive=True)
    x = sp.Matrix(x_symbols)

    # ---------------------------
    # Tạo bội số Lagrange λ (ineq) và v (eq)
    # ---------------------------
    if m_ineq > 0:
        lam_symbols = sp.symbols(f"l1:{m_ineq+1}", real=True, nonnegative=True)
        lam = sp.Matrix(lam_symbols)
    else:
        lam_symbols = []
        lam = sp.Matrix([])

    if m_eq > 0:
        v_symbols = sp.symbols(f"v1:{m_eq+1}", real=True)
        v = sp.Matrix(v_symbols)
    else:
        v_symbols = []
        v = sp.Matrix([])

    # ---------------------------
    # Hàm mục tiêu f(x) = sum x_i log x_i
    # ---------------------------
    f_expr = sum(xi * sp.log(xi) for xi in x_symbols)

    # ---------------------------
    # Xây g(x) <= 0, h(x) = 0
    # ---------------------------
    if m_ineq > 0:
        Aineq_sym = sp.Matrix(A_ineq_mat)
        bineq_sym = sp.Matrix(b_ineq_vec)
        g = Aineq_sym * x - bineq_sym          # A_ineq x - b_ineq <= 0
    else:
        g = sp.Matrix([])

    if m_eq > 0:
        Aeq_sym = sp.Matrix(A_eq_mat)
        beq_sym = sp.Matrix(b_eq_vec)
        h = Aeq_sym * x - beq_sym              # A_eq x - b_eq = 0
    else:
        h = sp.Matrix([])

    # ---------------------------
    # Lagrangian L(x, λ, v)
    # ---------------------------
    L = f_expr
    if m_ineq > 0:
        L += lam.dot(g)
    if m_eq > 0:
        L += v.dot(h)

    print("\n================ LAGRANGIAN =================")
    print("L(x, λ, v) =")
    sp.pprint(sp.simplify(L))

    # ---------------------------
    # STATIONARITY ∂L/∂x = 0
    # ---------------------------
    grad_L = [sp.simplify(sp.diff(L, xi)) for xi in x_symbols]
    print("\n================ STATIONARITY =================")
    for i, gi in enumerate(grad_L, start=1):
        print(f"∂L/∂x{i} = {gi} = 0")

    # ---------------------------
    # PRIMAL FEASIBILITY
    # ---------------------------
    print("\n================ PRIMAL FEASIBILITY =================")
    if m_ineq > 0:
        for i in range(m_ineq):
            print(f"g{i+1}(x) = {sp.simplify(g[i])} <= 0   (bất đẳng thức)")
    if m_eq > 0:
        for i in range(m_eq):
            print(f"h{i+1}(x) = {sp.simplify(h[i])} = 0   (đẳng thức)")

    # ---------------------------
    # DUAL FEASIBILITY
    # ---------------------------
    if m_ineq > 0:
        print("\n================ DUAL FEASIBILITY =================")
        for i in range(m_ineq):
            print(f"λ{i+1} >= 0")

    # ---------------------------
    # COMPLEMENTARY SLACKNESS
    # ---------------------------
    if m_ineq > 0:
        print("\n================ COMPLEMENTARY SLACKNESS =================")
        for i in range(m_ineq):
            print(f"λ{i+1} * g{i+1}(x) = λ{i+1}*({sp.simplify(g[i])}) = 0")

    # ---------------------------
    # XÂY HỆ PHƯƠNG TRÌNH KKT
    # ---------------------------
    eqs = []

    # stationarity
    eqs.extend(grad_L)

    # equality feasibility
    for i in range(m_eq):
        eqs.append(h[i])

    # complementary slackness
    for i in range(m_ineq):
        eqs.append(lam[i] * g[i])

    unknowns = list(x_symbols) + list(lam_symbols) + list(v_symbols)

    print("\n================ GIẢI HỆ KKT (Sympy) =================")
    sol = sp.solve(eqs, unknowns, dict=True, simplify=True)

    if not sol:
        print("⚠ Không tìm được nghiệm KKT (Sympy không giải được hệ).")
        return None

    sol = sol[0]

    print("\n===== MỘT NGHIỆM KKT TÌM ĐƯỢC =====")
    for var in unknowns:
        print(f"{var} = {sp.simplify(sol[var])}")

    # ---------------------------
    # TÍNH x*, f(x*)
    # ---------------------------
    x_star = np.array([float(sol[xi]) for xi in x_symbols])
    f_star = float(sum(x_star[i] * np.log(x_star[i]) for i in range(n)))

    print("\n================ OPTIMAL VALUE =================")
    print("x*   =", x_star)
    print("f(x*) =", f_star)

    return sol, x_star, f_star


In [ ]:
A = [
    [1,0,0],
    [0,2,0],
    [0,0,3]
]
# neu Ax >= b
b = [0.5, 1.0, 1.5]
A_ineq = -np.array(A)
b_ineq = -np.array(b)

A_eq = [[1,1,1]]
b_eq = [1.5]

solve_entropy_KKT(A_ineq=A_ineq, b_ineq=b_ineq,
                  A_eq=A_eq,     b_eq=b_eq)


In [ ]:
# neu Ax <= b
A_ineq = A
b_ineq = b

solve_entropy_KKT(A_ineq=A_ineq, b_ineq=b_ineq,
                  A_eq=A_eq,     b_eq=b_eq)


## **Câu 1 (2 điểm).**  
Xét bài toán tối ưu lồi sau đây:

$$
\min_{x \in \mathbb{R}^n} f(x) = \tfrac{1}{2}\lVert x\rVert_2^2 + c^\top x
\quad \text{s.t.} \quad a^\top x = b,
$$

trong đó $c \in \mathbb{R}^n$, $a \in \mathbb{R}^n$ và $b \in \mathbb{R}$ là các tham số đã cho.

---

### **Yêu cầu:**
1. Thiết lập hàm **Lagrange** của bài toán và suy ra **bài toán đối ngẫu (dual problem)**.  
2. Tìm nghiệm đối ngẫu tối ưu $\lambda^*$.  
3. Từ $\lambda^*$, suy ra nghiệm tối ưu của bài toán gốc $x^*$.  
4. Kiểm tra lại rằng nghiệm primal và dual cho cùng giá trị tối ưu.


In [ ]:
import sympy as sp
import numpy as np

def solve_primal_dual_quadratic(c_vec, a_vec, b_val):
    """
    Bài toán:
        minimize   f(x) = 1/2 ||x||^2 + c^T x
        subject to a^T x = b

    Input:
        c_vec : list / 1D np.array, kích thước n
        a_vec : list / 1D np.array, kích thước n
        b_val : số thực (float / int)

    In ra:
        1. Hàm Lagrange L(x, λ) và hàm đối ngẫu g(λ)
        2. λ* (dual optimum)
        3. x* (primal optimum)
        4. f(x*), g(λ*), và kiểm tra p* = g(λ*)
    """

    # ===================== 0. Chuẩn hoá dữ liệu =====================
    c_vec = np.array(c_vec, dtype=float).reshape(-1)
    a_vec = np.array(a_vec, dtype=float).reshape(-1)
    b_val = float(b_val)

    n = len(c_vec)
    if len(a_vec) != n:
        raise ValueError("c_vec và a_vec phải cùng chiều.")

    # ===================== 1. Tạo biến SymPy =====================
    x_symbols = sp.symbols(f"x1:{n+1}", real=True)
    x = sp.Matrix(x_symbols)           # vector x (sympy)
    c = sp.Matrix(c_vec)               # vector c (sympy)
    a = sp.Matrix(a_vec)               # vector a (sympy)
    b = sp.Float(b_val)                # scalar b
    lam = sp.Symbol("lam", real=True)  # Lagrange multiplier λ

    # ===================== 2. Hàm mục tiêu f(x) =====================
    # f(x) = 1/2 ||x||^2 + c^T x
    f_expr = sp.Rational(1, 2) * (x.dot(x)) + (c.dot(x))

    print("========== (1) HÀM MỤC TIÊU ==========")
    print("f(x) = 1/2 * ||x||^2 + c^T x =")
    sp.pprint(sp.simplify(f_expr))

    # ===================== 3. Ràng buộc a^T x = b =====================
    h = a.dot(x) - b   # = 0

    print("\nRàng buộc đẳng thức:")
    print("h(x) = a^T x - b = 0, với")
    print("a =", a_vec)
    print("b =", b_val)

    # ===================== 4. Hàm Lagrange L(x, λ) =====================
    L = f_expr + lam * h

    print("\n========== (2) HÀM LAGRANGE ==========")
    print("L(x, λ) = f(x) + λ (a^T x - b) =")
    sp.pprint(sp.simplify(L))

    # ===================== 5. Đạo hàm theo x: ∇_x L = 0 =====================
    grad_L = [sp.diff(L, xi) for xi in x_symbols]

    print("\nGradient theo x (điều kiện stationarity):")
    for i, gi in enumerate(grad_L, start=1):
        print(f"∂L/∂x{i} = {sp.simplify(gi)} = 0")

    # Solve ∂L/∂x = 0 để tìm x theo λ:
    sol_x_lambda = sp.solve(grad_L, list(x_symbols), dict=True)

    if not sol_x_lambda:
        print("\n⚠ Không giải được ∂L/∂x = 0 theo x.")
        return

    sol_x_lambda = sol_x_lambda[0]

    print("\n⇒ Nghiệm x(λ) thu được từ ∇_x L = 0:")
    for xi in x_symbols:
        print(f"{xi}(λ) = {sp.simplify(sol_x_lambda[xi])}")

    # ===================== 6. Hàm đối ngẫu g(λ) =====================
    # g(λ) = inf_x L(x, λ) = L(x(λ), λ)
    L_lambda = L.subs({xi: sol_x_lambda[xi] for xi in x_symbols})
    g_lambda = sp.simplify(L_lambda)

    print("\n========== (3) HÀM ĐỐI NGẪU g(λ) ==========")
    print("g(λ) = L(x(λ), λ) =")
    sp.pprint(g_lambda)

    print("\nBài toán đối ngẫu:")
    print("maximize g(λ)  over λ ∈ ℝ")

    # ===================== 7. Tìm λ* bằng cách giải g'(λ) = 0 =====================
    dg_dlam = sp.diff(g_lambda, lam)

    print("\nĐạo hàm g'(λ) = dg/dλ:")
    sp.pprint(sp.simplify(dg_dlam))
    print("Giải phương trình g'(λ) = 0:")

    lam_solutions = sp.solve(sp.Eq(dg_dlam, 0), lam, dict=True)

    if not lam_solutions:
        print("\n⚠ Không tìm được nghiệm dual λ*.")
        return

    lam_star_expr = lam_solutions[0][lam]
    lam_star_simplified = sp.simplify(lam_star_expr)

    print("\n========== (4) NGHIỆM ĐỐI NGẪU λ* ==========")
    print("λ* (dạng biểu thức) =")
    sp.pprint(lam_star_simplified)

    lam_star_val = float(lam_star_simplified)
    print(f"\nλ* (giá trị số) ≈ {lam_star_val:.6f}")

    # ===================== 8. Suy ra nghiệm primal x* =====================
    x_star_sym = sp.Matrix([sol_x_lambda[xi].subs(lam, lam_star_expr) for xi in x_symbols])
    x_star_simplified = sp.simplify(x_star_sym)

    print("\n========== (5) NGHIỆM PRIMAL x* ==========")
    print("x* (dạng biểu thức) =")
    sp.pprint(x_star_simplified)

    x_star_val = np.array([float(xx) for xx in x_star_simplified], dtype=float)
    print("\nx* (giá trị số) ≈", x_star_val)

    # ===================== 9. Giá trị primal f(x*) và dual g(λ*) =====================
    f_star_sym = f_expr.subs({x_symbols[i]: x_star_simplified[i] for i in range(n)})
    g_star_sym = g_lambda.subs(lam, lam_star_expr)

    f_star_simplified = sp.simplify(f_star_sym)
    g_star_simplified = sp.simplify(g_star_sym)

    print("\n========== (6) GIÁ TRỊ TỐI ƯU PRIMAL / DUAL ==========")
    print("f(x*) =")
    sp.pprint(f_star_simplified)
    print("\ng(λ*) =")
    sp.pprint(g_star_simplified)

    f_star_val = float(f_star_simplified)
    g_star_val = float(g_star_simplified)

    print(f"\nGiá trị số: f(x*) ≈ {f_star_val:.6f},  g(λ*) ≈ {g_star_val:.6f}")
    print("Hiệu f(x*) - g(λ*) ≈", float(f_star_val - g_star_val))

    print("\n⇒ Nếu hiệu ≈ 0 ➜ strong duality (primal = dual optimum).")

    return {
        "lambda_star_expr": lam_star_simplified,
        "lambda_star_val": lam_star_val,
        "x_star_expr": x_star_simplified,
        "x_star_val": x_star_val,
        "f_star_expr": f_star_simplified,
        "f_star_val": f_star_val,
        "g_star_expr": g_star_simplified,
        "g_star_val": g_star_val,
    }


# =========================================================
# 📌 VÍ DỤ CHẠY THỬ (bạn đổi c, a, b tuỳ ý)
# =========================================================
if __name__ == "__main__":
    # Ví dụ: n = 3
    c_vec = [1, -2]         # c ∈ R^3
    a_vec = [2, 1]         # a ∈ R^3
    b_val = 1.5                # b ∈ R

    result = solve_primal_dual_quadratic(c_vec, a_vec, b_val)


In [ ]:
import numpy as np

def dual_value(lambda_, c, a, b):
    """
    Tính giá trị hàm đối ngẫu g(λ) cho bài:
        min 1/2 ||x||^2 + c^T x  s.t. a^T x = b
    với x(λ) = -c - λ a
    """
    x_lam = -c - lambda_ * a
    # L(x, λ) = 1/2 ||x(λ)||^2 + c^T x(λ) + λ(a^T x(λ) - b)
    L = 0.5 * np.dot(x_lam, x_lam) + np.dot(c, x_lam) + lambda_ * (np.dot(a, x_lam) - b)
    return L

def solve_dual_1eq(c, a, b):
    """
    Giải đối ngẫu cho bài toán:
        min 1/2 ||x||^2 + c^T x
        s.t. a^T x = b

    Ta biết x(λ) = -c - λ a
    Thay vào ràng buộc a^T x = b:
        a^T(-c - λ a) = b
        -a^T c - λ a^T a = b
        λ a^T a = -(b + a^T c)
        λ* = -(b + a^T c) / (a^T a)

    Trả về:
        x_star, lambda_star, f_star
    """
    aTa = np.dot(a, a)          # a^T a (scalar)
    aTc = np.dot(a, c)          # a^T c (scalar)

    # λ*:
    lambda_star = -(b + aTc) / aTa

    # primal x*:
    x_star = -c - lambda_star * a

    # giá trị hàm mục tiêu primal f(x*)
    f_star = 0.5 * np.dot(x_star, x_star) + np.dot(c, x_star)

    return x_star, lambda_star, f_star

def check_primal_feasible(x, a, b, tol=1e-8):
    """
    Kiểm tra a^T x ≈ b
    """
    return abs(np.dot(a, x) - b) <= tol


if __name__ == "__main__":
    # =========================
    # VÍ DỤ CỤ THỂ CHO ĐỀ BÀI
    # =========================
    # Giả sử đề cho:
    # c = [1, -2, 3], a = [2, 1, -1], b = 4
    c = np.array([1., -2.])
    a = np.array([2., 1.])
    b = 1.5

    x_star, lam_star, f_star = solve_dual_1eq(c, a, b)

    print("✅ Nghiệm đối ngẫu tối ưu λ* =", lam_star)
    print("✅ Nghiệm primal suy ra x*   =", x_star)
    print("✅ Giá trị tối ưu f(x*)      =", f_star)

    # kiểm tra lại ràng buộc
    print("\nKiểm tra ràng buộc a^T x = b:")
    print("a^T x =", np.dot(a, x_star), " ~ b =", b)
    print("Feasible ?", check_primal_feasible(x_star, a, b))


In [ ]:
def lagrangian(x, lam, c, a, b):
    """
    L(x, λ) = 1/2 ||x||^2 + c^T x + λ(a^T x - b)

    Parameters
    ----------
    x : (n,) ndarray
        Biến primal.
    lam : float
        Biến đối ngẫu (λ).
    c : (n,) ndarray
        Vector c trong đề.
    a : (n,) ndarray
        Vector a trong ràng buộc a^T x = b.
    b : float
        Vế phải của ràng buộc.

    Returns
    -------
    float
        Giá trị Lagrangian tại (x, λ).
    """
    x = np.asarray(x)
    return 0.5 * np.dot(x, x) + np.dot(c, x) + lam * (np.dot(a, x) - b)
def dual_function(lam, c, a, b):
    """
    g(λ) = inf_x L(x, λ) = -1/2 ||c + λ a||^2 - b λ

    Parameters
    ----------
    lam : float or ndarray
        Giá trị λ cần tính g(λ).
    c, a : (n,) ndarray
    b : float

    Returns
    -------
    float or ndarray
        Giá trị hàm đối ngẫu g(λ).
    """
    v = c + lam * a   # c + λ a
    return -0.5 * np.dot(v, v) - b * lam

def solve_dual_optimal_lambda(c, a, b):
    """
    Tính nghiệm đối ngẫu tối ưu λ* = -(a^T c + b) / ||a||^2

    Parameters
    ----------
    c : (n,) ndarray
    a : (n,) ndarray
    b : float

    Returns
    -------
    float
        λ* nghiệm tối ưu của bài toán đối ngẫu.
    """
    num = np.dot(a, c) + b        # a^T c + b
    den = np.dot(a, a)            # ||a||^2
    return - num / den
def recover_primal_from_lambda(lam_star, c, a):
    """
    x* = - (c + λ* a)

    Parameters
    ----------
    lam_star : float
        Nghiệm đối ngẫu tối ưu.
    c, a : (n,) ndarray

    Returns
    -------
    (n,) ndarray
        Nghiệm primal tối ưu x*.
    """
    return - (c + lam_star * a)
def primal_value(x_star, c):
    """
    f(x*) = 1/2 ||x*||^2 + c^T x*

    Parameters
    ----------
    x_star : (n,) ndarray
    c : (n,) ndarray

    Returns
    -------
    float
    """
    return 0.5 * np.dot(x_star, x_star) + np.dot(c, x_star)


def check_strong_duality(c, a, b, tol=1e-8):
    """
    Tính λ*, x*, giá trị primal và dual rồi kiểm tra chúng trùng nhau.

    Parameters
    ----------
    c, a : (n,) ndarray
    b : float
    tol : float, optional
        Ngưỡng chấp nhận cho |p* - d*|.

    Returns
    -------
    dict
        {
          "lambda_star": ...,
          "x_star": ...,
          "primal_val": ...,
          "dual_val": ...,
          "feasible": True/False,   # a^T x* == b ?
          "strong_duality": True/False  # |p* - d*| <= tol
        }
    """
    lam_star = solve_dual_optimal_lambda(c, a, b)
    x_star = recover_primal_from_lambda(lam_star, c, a)
    p_val = primal_value(x_star, c)
    d_val = dual_function(lam_star, c, a, b)

    feasible = np.allclose(np.dot(a, x_star), b, atol=1e-8)
    strong = abs(p_val - d_val) <= tol

    return {
        "lambda_star": lam_star,
        "x_star": x_star,
        "primal_val": p_val,
        "dual_val": d_val,
        "feasible": feasible,
        "strong_duality": strong,
    }
if __name__ == "__main__":
    # ví dụ n = 3
    c = np.array([1., -2., 3.])
    a = np.array([2., 1., -1.])
    b = 4.

    res = check_strong_duality(c, a, b)
    print("λ* =", res["lambda_star"])
    print("x* =", res["x_star"])
    print("f(x*) =", res["primal_val"])
    print("g(λ*) =", res["dual_val"])
    print("a^T x* =", np.dot(a, res["x_star"]), " (phải =", b, ")")
    print("strong duality?", res["strong_duality"])


## **Câu 1 (2 điểm).**  
Xét bài toán tối ưu lồi dạng **Quadratic Programming** có ràng buộc tuyến tính đơn:

$$
\min_{x \in \mathbb{R}^2} \; f(x) = \tfrac{1}{2}x^\top P x + q^\top x
\quad \text{s.t.} \quad a^\top x = b,
$$

với các giá trị cụ thể:

$$
P =
\begin{bmatrix}
4 & 1 \\
1 & 2
\end{bmatrix},
\quad
q =
\begin{bmatrix}
-8 \\ -3
\end{bmatrix},
\quad
a =
\begin{bmatrix}
1 \\ 1
\end{bmatrix},
\quad
b = 3.
$$

---

### **Yêu cầu:**

1. Thiết lập **hàm Lagrange** của bài toán và suy ra **bài toán đối ngẫu (dual problem)**.  
   Viết biểu thức của $g(\lambda)$ dưới dạng tường minh theo $\lambda$.

2. Tính nghiệm đối ngẫu tối ưu $\lambda^*$ bằng cách giải phương trình đạo hàm của $g(\lambda)$.

3. Từ $\lambda^*$, suy ra nghiệm tối ưu của bài toán gốc $x^*$ bằng công thức:
   $$
   \nabla_x \mathcal{L}(x, \lambda^*) = 0 \quad \Rightarrow \quad x^* = -P^{-1}(q + \lambda^* a)
   $$

4. Tính và so sánh giá trị tối ưu **primal** $p^* = f(x^*)$ và **dual** $d^* = g(\lambda^*)$,  
   kiểm tra rằng $p^* = d^*$ để xác nhận **đối ngẫu mạnh (strong duality)**.

---

### 💡 Gợi ý công thức:

- Hàm Lagrange:
  $$
  \mathcal{L}(x, \lambda) = \tfrac{1}{2}x^\top P x + q^\top x + \lambda(a^\top x - b)
  $$

- Hàm đối ngẫu:
  $$
  g(\lambda) = -\tfrac{1}{2}(q+\lambda a)^\top P^{-1}(q+\lambda a) - b\lambda
  $$

- Nghiệm đối ngẫu tối ưu:
  $$
  \lambda^* = -\frac{a^\top P^{-1}q + b}{a^\top P^{-1}a}
  $$

- Nghiệm primal:
  $$
  x^* = -P^{-1}(q+\lambda^* a)
  $$


In [ ]:
import sympy as sp
import numpy as np

def solve_qp_one_equality(P_mat, q_vec, a_vec, b_val, numeric=True):
    """
    Bài toán QP lồi (1 ràng buộc đẳng thức):
        minimize   f(x) = 1/2 x^T P x + q^T x
        subject to a^T x = b

    Input:
        P_mat : (n,n) list/np.array
        q_vec : (n,)  list/np.array
        a_vec : (n,)  list/np.array
        b_val : scalar

    In ra:
        (1) L(x,λ), g(λ) tường minh theo λ
        (2) Giải g'(λ)=0 để lấy λ*
        (3) x* = -P^{-1}(q + λ*a)
        (4) p*=f(x*), d*=g(λ*) và kiểm tra strong duality
    """

    # ===================== 0) Chuẩn hoá dữ liệu =====================
    P_np = np.array(P_mat, dtype=float)
    q_np = np.array(q_vec, dtype=float).reshape(-1)
    a_np = np.array(a_vec, dtype=float).reshape(-1)
    b_val = float(b_val)

    n = q_np.shape[0]
    if P_np.shape != (n, n):
        raise ValueError("P_mat phải có shape (n,n) khớp với q_vec.")
    if a_np.shape[0] != n:
        raise ValueError("a_vec phải cùng chiều với q_vec.")

    # ===================== 1) Tạo biến SymPy =====================
    x_symbols = sp.symbols(f"x1:{n+1}", real=True)
    x = sp.Matrix(x_symbols)
    lam = sp.Symbol("lam", real=True)

    P = sp.Matrix(P_np)
    q = sp.Matrix(q_np)
    a = sp.Matrix(a_np)
    b = sp.Float(b_val)

    # ===================== 2) Hàm mục tiêu f(x) =====================
    f_expr = sp.Rational(1, 2) * (x.T * P * x)[0] + (q.T * x)[0]

    print("========== (1) HÀM MỤC TIÊU ==========")
    print("f(x) = 1/2 x^T P x + q^T x =")
    sp.pprint(sp.simplify(f_expr))

    # ===================== 3) Ràng buộc =====================
    h = (a.T * x)[0] - b
    print("\nRàng buộc: a^T x = b")
    print("a =", a_np)
    print("b =", b_val)

    # ===================== 4) Lagrangian =====================
    L = f_expr + lam * h
    print("\n========== (2) HÀM LAGRANGE ==========")
    print("L(x, λ) = 1/2 x^T P x + q^T x + λ(a^T x - b) =")
    sp.pprint(sp.simplify(L))

    # ===================== 5) Stationarity: ∇_x L = 0 =====================
    grad_L = [sp.diff(L, xi) for xi in x_symbols]

    print("\nĐiều kiện stationarity (∇_x L = 0):")
    for i, gi in enumerate(grad_L, start=1):
        print(f"∂L/∂x{i} = {sp.simplify(gi)} = 0")

    # Giải ∂L/∂x = 0 để lấy x(λ)
    sol_x_lambda = sp.solve(grad_L, list(x_symbols), dict=True)
    if not sol_x_lambda:
        raise ValueError("Không giải được hệ ∇_x L = 0 theo x.")
    sol_x_lambda = sol_x_lambda[0]

    print("\n⇒ Nghiệm x(λ) từ ∇_x L = 0:")
    for xi in x_symbols:
        print(f"{xi}(λ) = {sp.simplify(sol_x_lambda[xi])}")

    # ===================== 6) Dual function g(λ) = inf_x L(x,λ) =====================
    L_sub = L.subs({xi: sol_x_lambda[xi] for xi in x_symbols})
    g_lambda = sp.simplify(L_sub)

    print("\n========== (3) HÀM ĐỐI NGẪU g(λ) ==========")
    print("g(λ) = inf_x L(x,λ) = L(x(λ),λ) =")
    sp.pprint(g_lambda)

    # (Yêu cầu đề): đưa g(λ) tường minh theo λ dạng a2*λ^2 + a1*λ + a0
    g_poly = sp.Poly(sp.expand(g_lambda), lam)
    print("\nDạng tường minh theo λ (đa thức):")
    print("g(λ) = a2*λ^2 + a1*λ + a0, với:")
    print("a2 =", g_poly.coeffs()[0] if g_poly.degree() >= 2 else 0)
    if g_poly.degree() >= 1:
        print("a1 =", g_poly.coeffs()[1] if g_poly.degree() >= 2 else g_poly.coeffs()[0])
    print("a0 =", g_poly.TC())

    print("\nBài toán đối ngẫu: maximize g(λ) over λ ∈ ℝ")

    # ===================== 7) Tìm λ* bằng g'(λ)=0 =====================
    dg = sp.diff(g_lambda, lam)
    print("\nĐạo hàm g'(λ) =")
    sp.pprint(sp.simplify(dg))

    lam_solutions = sp.solve(sp.Eq(dg, 0), lam, dict=True)
    if not lam_solutions:
        raise ValueError("Không tìm được nghiệm λ* từ g'(λ)=0.")
    lam_star = sp.simplify(lam_solutions[0][lam])

    print("\n========== (4) NGHIỆM ĐỐI NGẪU λ* ==========")
    print("λ* =")
    sp.pprint(lam_star)

    # ===================== 8) Suy ra x* theo công thức đề bài =====================
    # x* = -P^{-1}(q + λ* a)
    # (tính theo SymPy cho đúng “đúng đề”)
    Pinv = P.inv()
    x_star = sp.simplify(-Pinv * (q + lam_star * a))

    print("\n========== (5) NGHIỆM PRIMAL x* ==========")
    print("x* = -P^{-1}(q + λ* a) =")
    sp.pprint(x_star)

    # ===================== 9) Tính p* và d* và so sánh =====================
    p_star = sp.simplify(f_expr.subs({x_symbols[i]: x_star[i] for i in range(n)}))
    d_star = sp.simplify(g_lambda.subs(lam, lam_star))

    print("\n========== (6) GIÁ TRỊ TỐI ƯU PRIMAL / DUAL ==========")
    print("p* = f(x*) =")
    sp.pprint(p_star)
    print("\nd* = g(λ*) =")
    sp.pprint(d_star)

    diff_expr = sp.simplify(p_star - d_star)
    print("\nKiểm tra strong duality: p* - d* =")
    sp.pprint(diff_expr)

    # kiểm tra ràng buộc
    feas = sp.simplify((a.T * x_star)[0] - b)
    print("\nKiểm tra ràng buộc: a^T x* - b =")
    sp.pprint(feas)

    result = {
        "g_lambda": g_lambda,
        "lambda_star": lam_star,
        "x_star": x_star,
        "p_star": p_star,
        "d_star": d_star,
        "p_minus_d": diff_expr,
        "feasibility": feas
    }

    if numeric:
        # số gần đúng
        lam_star_val = float(lam_star.evalf())
        x_star_val = np.array([float(x_star[i].evalf()) for i in range(n)], dtype=float)
        p_star_val = float(p_star.evalf())
        d_star_val = float(d_star.evalf())

        print("\n========== (7) GIÁ TRỊ SỐ GẦN ĐÚNG ==========")
        print(f"λ* ≈ {lam_star_val:.6f}")
        print("x* ≈", x_star_val)
        print(f"p* ≈ {p_star_val:.6f}")
        print(f"d* ≈ {d_star_val:.6f}")
        print(f"|p* - d*| ≈ {abs(p_star_val - d_star_val):.3e}")

        result.update({
            "lambda_star_val": lam_star_val,
            "x_star_val": x_star_val,
            "p_star_val": p_star_val,
            "d_star_val": d_star_val,
        })

    return result


# =========================================================
# ✅ CHẠY ĐÚNG ĐỀ BÀI CỦA BẠN
# =========================================================
if __name__ == "__main__":
    P = [[4, 1],
         [1, 2]]
    q = [-8, -3]
    a = [1, 1]
    b = 3

    solve_qp_one_equality(P, q, a, b, numeric=True)


In [ ]:
import numpy as np

# ================================================================
# 🧩 Câu 1 — Giải bài toán QP: 
# min 1/2 x^T P x + q^T x  s.t.  a^T x = b
# ================================================================


# ---------------------------------------------------------------
# 1️⃣  Định nghĩa hàm Lagrange và hàm đối ngẫu
# ---------------------------------------------------------------
def lagrangian(x, lam, P, q, a, b):
    """
    Hàm Lagrange: L(x, λ) = 1/2 x^T P x + q^T x + λ (a^T x - b)
    
    Parameters
    ----------
    x : (n,) ndarray - biến primal
    lam : float      - biến dual (λ)
    P : (n,n) ndarray - ma trận xác định dương
    q : (n,) ndarray
    a : (n,) ndarray
    b : float
    
    Returns
    -------
    float
        Giá trị Lagrangian tại (x, λ)
    """
    return 0.5 * x.T @ P @ x + q.T @ x + lam * (a.T @ x - b)


def dual_function(lam, P, q, a, b):
    """
    Hàm đối ngẫu: g(λ) = -1/2 (q + λ a)^T P^{-1} (q + λ a) - bλ
    """
    P_inv = np.linalg.inv(P)
    v = q + lam * a
    return -0.5 * v.T @ P_inv @ v - b * lam


# ---------------------------------------------------------------
# 2️⃣  Giải nghiệm đối ngẫu λ*
# ---------------------------------------------------------------
def solve_lambda_star(P, q, a, b):
    """
    λ* = - (a^T P^{-1} q + b) / (a^T P^{-1} a)
    """
    P_inv = np.linalg.inv(P)
    num = a.T @ P_inv @ q + b
    den = a.T @ P_inv @ a
    lam_star = - num / den
    return lam_star.item()  # Trả về số thực


# ---------------------------------------------------------------
# 3️⃣  Suy ra nghiệm primal x*
# ---------------------------------------------------------------
def solve_x_star(P, q, a, lam_star):
    """
    x* = - P^{-1} (q + λ* a)
    """
    P_inv = np.linalg.inv(P)
    return - P_inv @ (q + lam_star * a)


# ---------------------------------------------------------------
# 4️⃣  Tính và so sánh giá trị tối ưu primal & dual
# ---------------------------------------------------------------
def primal_value(x, P, q):
    """
    f(x) = 1/2 x^T P x + q^T x
    """
    return 0.5 * x.T @ P @ x + q.T @ x


def dual_value(lam_star, P, q, a, b):
    """
    d(λ*) = g(λ*)
    """
    return dual_function(lam_star, P, q, a, b)


def check_strong_duality(P, q, a, b, tol=1e-8):
    """
    Tính λ*, x*, p*, d* và kiểm tra strong duality.
    """
    lam_star = solve_lambda_star(P, q, a, b)
    x_star = solve_x_star(P, q, a, lam_star)
    p_star = primal_value(x_star, P, q)
    d_star = dual_value(lam_star, P, q, a, b)
    feasible = np.allclose(a.T @ x_star, b, atol=1e-6)
    strong = np.abs(p_star - d_star) < tol
    
    return {
        "lambda_star": lam_star,
        "x_star": x_star,
        "p_star": float(p_star),
        "d_star": float(d_star),
        "feasible": feasible,
        "strong_duality": strong
    }


# ---------------------------------------------------------------
# 🚀  Chạy ví dụ cụ thể trong đề
# ---------------------------------------------------------------
P = np.array([[4, 1],
              [1, 2]], dtype=float)
q = np.array([-8, -3], dtype=float)
a = np.array([1, 1], dtype=float)
b = 3.0

result = check_strong_duality(P, q, a, b)

print("λ* =", result["lambda_star"])
print("x* =", result["x_star"])
print("p* =", result["p_star"])
print("d* =", result["d_star"])
print("a^T x* =", a.T @ result["x_star"], " (phải =", b, ")")
print("Strong duality:", result["strong_duality"])


## **Câu 1 (2 điểm).**  
Xét bài toán tối ưu lồi dạng **Quadratic Programming** có ràng buộc tuyến tính chứa thêm một biến vô hướng:

$$
\min_{x \in \mathbb{R}^2,\; y \in \mathbb{R}} \; F(x,y) = \tfrac{1}{2}x^\top P x + q^\top x + r y
\quad \text{s.t.} \quad a^\top x + \alpha y = b,
$$

với các giá trị cụ thể:

$$
P =
\begin{bmatrix}
4 & 1 \\
1 & 2
\end{bmatrix},
\quad
q =
\begin{bmatrix}
-8 \\ -3
\end{bmatrix},
\quad
r = 2,
\quad
a =
\begin{bmatrix}
1 \\ 1
\end{bmatrix},
\quad
\alpha = 3,
\quad
b = 3.
$$

---

### **Yêu cầu:**

1. Thiết lập **hàm Lagrange** của bài toán và suy ra **bài toán đối ngẫu** theo biến đối ngẫu $\lambda \in \mathbb{R}$.  
   Viết rõ điều kiện để hàm đối ngẫu $g(\lambda)$ là hữu hạn.

2. Tối ưu theo $x$ và theo $y$ để tìm biểu thức tường minh của $g(\lambda)$.

3. Tìm nghiệm đối ngẫu tối ưu $\lambda^*$, sau đó suy ra nghiệm primal tối ưu $(x^*, y^*)$.

4. Tính và so sánh giá trị tối ưu **primal** $p^* = F(x^*, y^*)$ và **dual** $d^* = g(\lambda^*)$,  
   kết luận về **đối ngẫu mạnh** trong bài toán này.

---

### 💡 Gợi ý công thức:

- Hàm Lagrange:

  $$
  \mathcal{L}(x,y,\lambda)
  = \tfrac{1}{2}x^\top P x + q^\top x + r y
    + \lambda (a^\top x + \alpha y - b).
  $$

- Điều kiện cực tiểu theo $x$:

  $$
  \nabla_x \mathcal{L} = P x + q + \lambda a = 0
  \quad \Rightarrow \quad
  x(\lambda) = - P^{-1}(q + \lambda a).
  $$

- Điều kiện cực tiểu theo $y$:

  $$
  \frac{\partial \mathcal{L}}{\partial y} = r + \lambda \alpha = 0
  \quad \Rightarrow \quad
  y(\lambda) = \text{tìm được trực tiếp từ } r + \lambda \alpha = 0.
  $$

- Thay $x(\lambda)$ và $y(\lambda)$ vào $\mathcal{L}$ để thu được $g(\lambda)$, rồi giải $\max_\lambda g(\lambda)$.


In [ ]:
import numpy as np

# ================================================================
# 🧩 Bài toán:
# min_{x∈R^2, y∈R}  F(x,y) = 1/2 x^T P x + q^T x + r y
# s.t.               a^T x + α y = b
# ================================================================


# ---------------------------------------------------------------
# 1️⃣ Hàm Lagrange và hàm đối ngẫu
# ---------------------------------------------------------------
def lagrangian(x, y, lam, P, q, r, a, alpha, b):
    """
    L(x, y, λ) = 1/2 x^T P x + q^T x + r y + λ(a^T x + α y - b)
    """
    return 0.5 * x.T @ P @ x + q.T @ x + r * y + lam * (a.T @ x + alpha * y - b)


def dual_function(lam, P, q, r, a, alpha, b):
    """
    g(λ) = inf_{x,y} L(x, y, λ)
          = -1/2 (q + λ a)^T P^{-1} (q + λ a) - bλ - (r + λ α)^2 * 0 (vì y xác định trực tiếp)
    """
    P_inv = np.linalg.inv(P)
    v = q + lam * a
    y_term = r + lam * alpha
    # y được xác định trực tiếp từ r + λ α = 0  ⇒ chỉ hữu hạn khi y_term = 0
    if abs(y_term) > 1e-10:
        return -np.inf  # g(λ) = -∞ nếu điều kiện không thỏa
    return -0.5 * v.T @ P_inv @ v - b * lam


# ---------------------------------------------------------------
# 2️⃣ Giải nghiệm đối ngẫu λ*
# ---------------------------------------------------------------
def solve_lambda_star(P, q, r, a, alpha, b):
    """
    Do ∂L/∂y = 0 → r + λ α = 0 ⇒ λ = -r/α
    """
    return -r / alpha


# ---------------------------------------------------------------
# 3️⃣ Từ λ* suy ra nghiệm primal (x*, y*)
# ---------------------------------------------------------------
def solve_primal_xy(P, q, r, a, alpha, b, lam_star):
    """
    x* = -P^{-1}(q + λ* a)
    y* = -(r + λ* α)/α  nhưng ở đây r + λ* α = 0 ⇒ y* được tìm từ ràng buộc a^T x + α y = b
    """
    P_inv = np.linalg.inv(P)
    x_star = -P_inv @ (q + lam_star * a)
    y_star = (b - a.T @ x_star) / alpha
    return x_star, float(y_star)


# ---------------------------------------------------------------
# 4️⃣ Hàm tính giá trị primal và dual
# ---------------------------------------------------------------
def primal_value(x, y, P, q, r):
    return 0.5 * x.T @ P @ x + q.T @ x + r * y


def dual_value(lam_star, P, q, r, a, alpha, b):
    return dual_function(lam_star, P, q, r, a, alpha, b)


# ---------------------------------------------------------------
# 5️⃣ Gói hàm kiểm tra toàn bộ
# ---------------------------------------------------------------
def check_strong_duality_xy(P, q, r, a, alpha, b, tol=1e-8):
    lam_star = solve_lambda_star(P, q, r, a, alpha, b)
    x_star, y_star = solve_primal_xy(P, q, r, a, alpha, b, lam_star)
    p_star = primal_value(x_star, y_star, P, q, r)
    d_star = dual_value(lam_star, P, q, r, a, alpha, b)
    feasible = np.allclose(a.T @ x_star + alpha * y_star, b, atol=1e-6)
    strong = abs(p_star - d_star) <= tol

    return {
        "lambda_star": float(lam_star),
        "x_star": x_star,
        "y_star": float(y_star),
        "p_star": float(p_star),
        "d_star": float(d_star),
        "feasible": feasible,
        "strong_duality": strong,
    }


# ---------------------------------------------------------------
# 🚀 Chạy ví dụ trong đề
# ---------------------------------------------------------------
P = np.array([[4, 1],
              [1, 2]], dtype=float)
q = np.array([-8, -3], dtype=float)
r = 2.0
a = np.array([1, 1], dtype=float)
alpha = 3.0
b = 3.0

result = check_strong_duality_xy(P, q, r, a, alpha, b)

print("λ* =", result["lambda_star"])
print("x* =", result["x_star"])
print("y* =", result["y_star"])
print("p* =", result["p_star"])
print("d* =", result["d_star"])
print("a^T x* + α y* =", a.T @ result["x_star"] + alpha * result["y_star"], " (phải =", b, ")")
print("Strong duality:", result["strong_duality"])


## **Câu 1 (2 điểm).**  
Xét bài toán tối ưu lồi dạng **Quadratic Programming mở rộng** có ràng buộc tuyến tính đơn:

$$
\min_{x,y \in \mathbb{R}} \; f(x,y) = \tfrac{1}{2}
\begin{bmatrix}x \\ y\end{bmatrix}^\top
\begin{bmatrix}
4 & 1 \\
1 & 2
\end{bmatrix}
\begin{bmatrix}x \\ y\end{bmatrix}
+
\begin{bmatrix}-8 & -3\end{bmatrix}
\begin{bmatrix}x \\ y\end{bmatrix}
+ 2y
$$

với ràng buộc tuyến tính:

$$
x + 2y = 4.
$$

---

### **Yêu cầu:**

1. Thiết lập **hàm Lagrange** của bài toán và suy ra **bài toán đối ngẫu (dual problem)** theo biến đối ngẫu $\lambda \in \mathbb{R}$.  
   Viết biểu thức tường minh của hàm đối ngẫu $g(\lambda)$.

2. Tối ưu theo $x$ và $y$ để tìm nghiệm đối ngẫu tối ưu $\lambda^*$.

3. Từ $\lambda^*$, suy ra nghiệm primal tối ưu $(x^*, y^*)$ theo công thức:
   $$
   \nabla_{(x,y)} \mathcal{L}(x,y,\lambda^*) = 0
   \quad \Rightarrow \quad
   \begin{bmatrix}x^* \\ y^*\end{bmatrix}
   = -P^{-1} (q + \lambda^* a),
   $$
   trong đó
   $$
   P =
   \begin{bmatrix}
   4 & 1 \\
   1 & 2
   \end{bmatrix},
   \quad
   q =
   \begin{bmatrix}
   -8 \\ -1
   \end{bmatrix},
   \quad
   a =
   \begin{bmatrix}
   1 \\ 2
   \end{bmatrix},
   \quad
   b = 4.
   $$

4. Tính và so sánh giá trị tối ưu **primal** $p^* = f(x^*,y^*)$ và **dual** $d^* = g(\lambda^*)$.  
   Kiểm tra rằng $p^* = d^*$ để xác nhận **đối ngẫu mạnh (strong duality)**.

---

### 💡 Gợi ý công thức:

- Hàm Lagrange:
  $$
  \mathcal{L}(x,y,\lambda)
  = \tfrac{1}{2}
  \begin{bmatrix}x \\ y\end{bmatrix}^\top
  P
  \begin{bmatrix}x \\ y\end{bmatrix}
  + q^\top
  \begin{bmatrix}x \\ y\end{bmatrix}
  + \lambda(a_1 x + a_2 y - b)
  $$

- Hàm đối ngẫu:
  $$
  g(\lambda)
  = -\tfrac{1}{2}(q+\lambda a)^\top P^{-1}(q+\lambda a) - b\lambda
  $$

- Nghiệm đối ngẫu tối ưu:
  $$
  \lambda^* = -\frac{a^\top P^{-1}q + b}{a^\top P^{-1}a}
  $$

- Nghiệm primal:
  $$
  \begin{bmatrix}x^* \\ y^*\end{bmatrix}
  = -P^{-1}(q+\lambda^* a)
  $$


In [ ]:
import numpy as np

# ============================================================
# 🧩 Bài toán:
# min_{x,y ∈ R}  f(x,y) = 1/2 [x y]^T P [x y] + q^T [x y] + const
# s.t.            a^T [x y] = b
# ============================================================


# ------------------------------------------------------------
# 1️⃣ Hàm Lagrange và hàm đối ngẫu
# ------------------------------------------------------------
def lagrangian(x, y, lam, P, q, a, b):
    """
    L(x,y,λ) = 1/2 [x y]^T P [x y] + q^T [x y] + λ(a^T [x y] - b)
    """
    vec = np.array([x, y])
    return 0.5 * vec.T @ P @ vec + q.T @ vec + lam * (a.T @ vec - b)


def dual_function(lam, P, q, a, b):
    """
    g(λ) = -1/2 (q + λ a)^T P^{-1} (q + λ a) - b λ
    """
    P_inv = np.linalg.inv(P)
    v = q + lam * a
    return -0.5 * v.T @ P_inv @ v - b * lam


# ------------------------------------------------------------
# 2️⃣ Giải nghiệm đối ngẫu λ*
# ------------------------------------------------------------
def solve_lambda_star(P, q, a, b):
    """
    λ* = - (a^T P^{-1} q + b) / (a^T P^{-1} a)
    """
    P_inv = np.linalg.inv(P)
    num = a.T @ P_inv @ q + b
    den = a.T @ P_inv @ a
    lam_star = -num / den
    return lam_star.item()  # đảm bảo là số thực (scalar)


# ------------------------------------------------------------
# 3️⃣ Từ λ* suy ra nghiệm primal (x*, y*)
# ------------------------------------------------------------
def solve_primal_xy(P, q, a, lam_star):
    """
    [x* y*]^T = - P^{-1} (q + λ* a)
    """
    P_inv = np.linalg.inv(P)
    xy_star = -P_inv @ (q + lam_star * a)
    return xy_star


# ------------------------------------------------------------
# 4️⃣ Tính giá trị primal và dual
# ------------------------------------------------------------
def primal_value(xy_star, P, q):
    """
    f(x*) = 1/2 x*^T P x* + q^T x*
    """
    return 0.5 * xy_star.T @ P @ xy_star + q.T @ xy_star


def dual_value(lam_star, P, q, a, b):
    """
    d(λ*) = g(λ*)
    """
    return dual_function(lam_star, P, q, a, b)


# ------------------------------------------------------------
# 5️⃣ Hàm kiểm tra và in kết quả
# ------------------------------------------------------------
def check_strong_duality(P, q, a, b, tol=1e-8):
    """
    Giải toàn bộ bài toán: λ*, (x*,y*), p*, d*, kiểm tra strong duality
    """
    lam_star = solve_lambda_star(P, q, a, b)
    xy_star = solve_primal_xy(P, q, a, lam_star)
    p_star = primal_value(xy_star, P, q)
    d_star = dual_value(lam_star, P, q, a, b)
    feasible = np.allclose(a.T @ xy_star, b, atol=1e-6)
    strong = abs(p_star - d_star) <= tol

    return {
        "lambda_star": float(lam_star),
        "x_star": float(xy_star[0]),
        "y_star": float(xy_star[1]),
        "p_star": float(p_star),
        "d_star": float(d_star),
        "feasible": feasible,
        "strong_duality": strong,
    }


# ------------------------------------------------------------
# 🚀 DỮ LIỆU BÀI TOÁN (đúng theo đề)
# ------------------------------------------------------------
P = np.array([[4, 1],
              [1, 2]], dtype=float)
q = np.array([-8, -1], dtype=float)
a = np.array([1, 2], dtype=float)
b = 4.0

# ------------------------------------------------------------
# ⚙️ Giải và in kết quả
# ------------------------------------------------------------
res = check_strong_duality(P, q, a, b)

print("λ* =", res["lambda_star"])
print("x* =", res["x_star"])
print("y* =", res["y_star"])
print("p* =", res["p_star"])
print("d* =", res["d_star"])
print("a^T [x* y*] =", a.T @ np.array([res["x_star"], res["y_star"]]), " (phải =", b, ")")
print("Strong duality:", res["strong_duality"])


## **Câu 1 (2 điểm).**  
Xét bài toán **tối ưu lồi phi tuyến** (mở rộng từ Quadratic Programming) sau:

$$
\min_{x, y \in \mathbb{R}} \; f(x,y)
= (x - 1)^2 + (y - 2)^2 + e^{0.2x} + \sin(y)
$$

với các **ràng buộc tuyến tính và phi tuyến** sau:

$$
\begin{cases}
x + y = 3, \\
x - 2y \le 1, \\
x \ge 0.
\end{cases}
$$

---

### **Yêu cầu:**

1. Thiết lập **hàm Lagrange mở rộng**:
   $$
   \mathcal{L}(x, y, \lambda_1, \lambda_2, \lambda_3)
   = f(x, y)
   + \lambda_1(x + y - 3)
   + \lambda_2(x - 2y - 1)
   - \lambda_3 x,
   $$
   trong đó:
   - $\lambda_1$ là nhân tử Lagrange cho ràng buộc bằng,
   - $\lambda_2, \lambda_3 \ge 0$ cho các ràng buộc bất đẳng thức.

2. Viết **hệ điều kiện KKT** đầy đủ:
   - Điều kiện dừng (stationarity),
   - Tính khả thi (primal feasibility),
   - Tính khả thi đối ngẫu (dual feasibility),
   - Điều kiện bổ sung (complementary slackness).

3. Giải hệ KKT để tìm nghiệm tối ưu $(x^*, y^*, \lambda_1^*, \lambda_2^*, \lambda_3^*)$.

4. Tính giá trị tối ưu của hàm mục tiêu:
   $$
   p^* = f(x^*, y^*).
   $$

---

### 💡 Gợi ý công thức:
- Điều kiện dừng:
  $$
  \begin{cases}
  \frac{\partial \mathcal{L}}{\partial x} = 2(x - 1) + 0.2e^{0.2x} + \lambda_1 + \lambda_2 - \lambda_3 = 0, \\
  \frac{\partial \mathcal{L}}{\partial y} = 2(y - 2) + \cos(y) + \lambda_1 - 2\lambda_2 = 0.
  \end{cases}
  $$

- Điều kiện khả thi:
  $$
  \begin{cases}
  x + y - 3 = 0, \\
  x - 2y - 1 \le 0, \\
  x \ge 0.
  \end{cases}
  $$

- Điều kiện đối ngẫu:
  $$
  \lambda_2 \ge 0, \quad \lambda_3 \ge 0.
  $$

- Bổ sung:
  $$
  \lambda_2(x - 2y - 1) = 0, \quad \lambda_3 x = 0.
  $$

---

### 📘 Gợi ý hướng giải:
- Thử xét các **trường hợp hoạt hóa ràng buộc** (active constraints):
  - Nếu $x = 0$, tìm $(y, \lambda_1, \lambda_2)$.
  - Nếu $x - 2y = 1$, tìm $(x, y, \lambda_1, \lambda_3)$.
  - Nếu chỉ có ràng buộc bằng hoạt hóa, kiểm tra tính khả thi và cực trị.
- So sánh các giá trị $f(x,y)$ giữa các trường hợp để chọn nghiệm tối ưu toàn cục.


In [ ]:
import sympy as sp
import math

# ============================================================
# 1. Định nghĩa bài toán
# ============================================================
# Biến
x, y = sp.symbols('x y', real=True)
lam1, lam2, lam3 = sp.symbols('lam1 lam2 lam3', real=True, nonnegative=False)  # lam2, lam3 sẽ kiểm tra ≥0 sau

# Hàm mục tiêu: f(x,y) = (x-1)^2 + (y-2)^2 + e^{0.2x} + sin(y)
f = (x - 1)**2 + (y - 2)**2 + sp.exp(0.2 * x) + sp.sin(y)

# Ràng buộc:
g1 = x + y - 3           # = 0
g2 = x - 2*y - 1         # ≤ 0
g3 = -x                  # x ≥ 0  ⇔  -x ≤ 0

# Lagrangian
L = f + lam1 * g1 + lam2 * g2 + lam3 * g3

# Đạo hàm Lagrangian
dL_dx = sp.diff(L, x)
dL_dy = sp.diff(L, y)


# ============================================================
# 2. Hàm tính f(x,y) thuần (dùng để so sánh)
# ============================================================
def f_val(xv, yv):
    return (xv - 1)**2 + (yv - 2)**2 + math.exp(0.2 * xv) + math.sin(yv)


# ============================================================
# 3. CASE 1: Chỉ ràng buộc bằng hoạt hóa
#    x + y = 3
#    x - 2y < 1  → lam2 = 0
#    x > 0       → lam3 = 0
# ============================================================
def solve_case1():
    eq1 = sp.Eq(dL_dx.subs({lam2: 0, lam3: 0}), 0)
    eq2 = sp.Eq(dL_dy.subs({lam2: 0, lam3: 0}), 0)
    eq3 = sp.Eq(g1, 0)  # x + y = 3

    # ẩn: x, y, lam1
    sol = sp.nsolve(
        [
            eq1.subs(lam1, lam1),
            eq2.subs(lam1, lam1),
            eq3
        ],
        [x, y, lam1],
        [1.0, 2.0, 0.0]   # đoán ban đầu
    )
    xv, yv, lam1v = [float(s) for s in sol]

    # kiểm tra tính khả thi của 2 bất đẳng thức
    g2v = xv - 2*yv - 1
    g3v = -xv
    feasible = (g2v < 1e-6) and (g3v <= 1e-6)  # x-2y-1 ≤ 0 và x ≥ 0

    return {
        "case": "1: only equality active",
        "x": xv,
        "y": yv,
        "lam1": lam1v,
        "lam2": 0.0,
        "lam3": 0.0,
        "feasible": feasible,
        "f": f_val(xv, yv)
    }


# ============================================================
# 4. CASE 2: ràng buộc bằng + ràng buộc x - 2y = 1 (active)
#    x + y = 3
#    x - 2y = 1
#    x ≥ 0  → nếu x>0 thì lam3=0
# ============================================================
def solve_case2():
    # hệ:
    # dL/dx = 0
    # dL/dy = 0
    # g1 = 0
    # g2 = 0
    # với lam3 = 0 (tạm giả sử không active)
    eq1 = sp.Eq(dL_dx.subs({lam3: 0}), 0)
    eq2 = sp.Eq(dL_dy.subs({lam3: 0}), 0)
    eq3 = sp.Eq(g1, 0)
    eq4 = sp.Eq(g2, 0)

    sol = sp.nsolve(
        [eq1, eq2, eq3, eq4],
        [x, y, lam1, lam2],
        [1.0, 1.0, 0.0, 0.5]
    )
    xv, yv, lam1v, lam2v = [float(s) for s in sol]

    # kiểm tra x ≥ 0
    g3v = -xv
    feasible = (g3v <= 1e-6) and (lam2v >= -1e-6)  # lam2 ≥ 0

    return {
        "case": "2: equality + (x - 2y = 1) active",
        "x": xv,
        "y": yv,
        "lam1": lam1v,
        "lam2": lam2v,
        "lam3": 0.0,
        "feasible": feasible,
        "f": f_val(xv, yv)
    }


# ============================================================
# 5. CASE 3: ràng buộc bằng + ràng buộc x = 0 active
#    x = 0  → g3 = 0
#    x + y = 3  → y = 3
#    check x - 2y -1 ≤ 0
# ============================================================
def solve_case3():
    xv = 0.0
    yv = 3.0
    # Khi x=0, y=3 → ràng buộc thứ 2: 0 - 6 - 1 = -7 ≤ 0 → ok
    # dL/dx = 0 → 2(x-1)+0.2e^{0.2x} + lam1 + lam2 - lam3 = 0
    # nhưng ở đây ta đang xét x=0 active → lam3 ≥ 0 và lam3 * x = 0 → không giúp tìm lam ngay
    # Ta có thể giải ngược lam1, lam2 từ 2 PT stationarity
    eq1 = sp.Eq(2*(xv - 1) + 0.2*math.exp(0.2*xv) + lam1 + lam2 - lam3, 0)
    eq2 = sp.Eq(2*(yv - 2) + math.cos(yv) + lam1 - 2*lam2, 0)
    # x=0 active ⇒ lam3 ≥ 0, còn lam3 ta coi là ẩn, ta đặt lam3 = t và giải lam1, lam2 theo t
    # để đơn giản: đặt lam3 = 0, rồi kiểm tra sau
    sol = sp.solve(
        [
            eq1.subs(lam3, 0),
            eq2
        ],
        (lam1, lam2),
        dict=True
    )
    lam1v = float(sol[0][lam1])
    lam2v = float(sol[0][lam2])

    feasible = (xv >= -1e-6) and ((xv - 2*yv - 1) <= 1e-6) and (lam2v >= -1e-6)
    return {
        "case": "3: equality + x=0 active",
        "x": xv,
        "y": yv,
        "lam1": lam1v,
        "lam2": lam2v,
        "lam3": 0.0,
        "feasible": feasible,
        "f": f_val(xv, yv)
    }


# ============================================================
# 6. Chạy cả 3 case và chọn nghiệm tốt nhất khả thi
# ============================================================
sols = []
try:
    sols.append(solve_case1())
except Exception as e:
    print("Case 1 fail:", e)

try:
    sols.append(solve_case2())
except Exception as e:
    print("Case 2 fail:", e)

try:
    sols.append(solve_case3())
except Exception as e:
    print("Case 3 fail:", e)

# lọc nghiệm khả thi
feasible_sols = [s for s in sols if s["feasible"]]

# chọn nghiệm có f nhỏ nhất
best = min(feasible_sols, key=lambda s: s["f"])

print("=== Các nghiệm tìm được ===")
for s in sols:
    print(s)

print("\n=== Nghiệm tốt nhất (theo f) ===")
print(best)


## **Câu 1 (2 điểm).**  
Xét bài toán tối ưu lồi dạng **Quadratic + Norm** sau:

$$
\min_{x \in \mathbb{R}^2} \; f(x)
= \tfrac{1}{2}\lVert x\rVert_2^2 + c^\top x + \lVert A x + b \rVert_2
\quad \text{s.t.} \quad a^\top x = d,
$$

trong đó

$$
A =
\begin{bmatrix}
1 & 0 \\
0 & 2
\end{bmatrix}, \quad
b =
\begin{bmatrix}
-1 \\ 1
\end{bmatrix}, \quad
c =
\begin{bmatrix}
-2 \\ 1
\end{bmatrix}, \quad
a =
\begin{bmatrix}
1 \\ 1
\end{bmatrix}, \quad
d = 1.
$$

---

### 1. Thiết lập hàm Lagrange và suy ra bài toán đối ngẫu

Hàm Lagrange (chỉ có ràng buộc bằng) là

$$
\mathcal{L}(x, \lambda)
= \tfrac{1}{2}\lVert x\rVert_2^2 + c^\top x + \lVert A x + b \rVert_2 + \lambda (a^\top x - d),
\quad \lambda \in \mathbb{R}.
$$

Hàm đối ngẫu được định nghĩa như sau:

$$
g(\lambda) = \inf_{x \in \mathbb{R}^2} \; 
\Big(
\tfrac{1}{2}\lVert x\rVert_2^2 + c^\top x + \lVert A x + b \rVert_2 + \lambda (a^\top x - d)
\Big).
$$

Vì trong mục tiêu có hạng $\lVert A x + b \rVert_2$, ta có thể viết lại bằng hàm liên hợp của chuẩn 2:

$$
\lVert A x + b \rVert_2
= \max_{\lVert z \rVert_2 \le 1} z^\top (A x + b).
$$

Thay vào Lagrangian, ta thu được dạng đối ngẫu “lồng”:

$$
g(\lambda)
= \inf_x \max_{\lVert z \rVert \le 1}
\Big(
\tfrac{1}{2}\lVert x\rVert_2^2
+ (c + A^\top z + \lambda a)^\top x
+ z^\top b
- \lambda d
\Big).
$$

Đổi thứ tự (do bài toán lồi và tập $\{z: \|z\|\le1\}$ compact):

$$
g(\lambda)
= \max_{\lVert z \rVert \le 1}
\Big(
\inf_x \big[
\tfrac{1}{2}\lVert x\rVert_2^2 + (c + A^\top z + \lambda a)^\top x
\big]
+ z^\top b - \lambda d
\Big).
$$

Với phần “inf theo \(x\)” (đây là quadratic thuần), ta có nghiệm đóng:

- nghiệm theo \(x\):  
  $$
  x(\lambda, z) = - (c + A^\top z + \lambda a),
  $$
  vì $\nabla_x = x + c + A^\top z + \lambda a = 0$.

- giá trị inf:
  $$
  \inf_x \big[
  \tfrac{1}{2}\lVert x\rVert_2^2 + (c + A^\top z + \lambda a)^\top x
  \big]
  = - \tfrac{1}{2} \lVert c + A^\top z + \lambda a \rVert_2^2.
  $$

Vậy hàm đối ngẫu có dạng:

$$
g(\lambda)
= \max_{\lVert z \rVert \le 1}
\Big(
- \tfrac{1}{2} \lVert c + A^\top z + \lambda a \rVert_2^2
+ z^\top b
- \lambda d
\Big).
$$

Bài toán đối ngẫu là:

$$
\max_{\lambda \in \mathbb{R}} \; g(\lambda)
= \max_{\lambda \in \mathbb{R}} \max_{\lVert z \rVert \le 1}
\Big(
- \tfrac{1}{2} \lVert c + A^\top z + \lambda a \rVert_2^2
+ z^\top b
- \lambda d
\Big).
$$

(hoặc đổi thứ tự max nếu bạn muốn: \(\max_{\|z\|\le1} \max_\lambda (\cdots)\)).

---

### 2. Tìm nghiệm đối ngẫu tối ưu \(\lambda^*\)

Từ biểu thức trên, để tìm \(\lambda^*\) ta phải tối ưu đồng thời theo \(\lambda\) và \(z\) (với \(\|z\|\le1\)).  
Có hai cách:

- Cách 1 (lý thuyết): cố định \(z\), tối ưu theo \(\lambda\) trước. Khi đó ta có hàm bậc hai theo \(\lambda\) và tìm được \(\lambda(z)\) theo công thức giải phương trình bậc hai.
- Cách 2 (thực hành): duyệt / tối ưu số (gradient / grid nhỏ) trên miền \(\|z\|\le1\), với mỗi \(z\) giải \(\lambda\), rồi chọn giá trị lớn nhất.

Vì đề đang ở dạng “sinh bài để code”, ta có thể ghi công thức tối ưu theo \(\lambda\) khi \(z\) cố định:

Đặt
$$
u(z) = c + A^\top z, \quad \text{khi đó} \quad
g(\lambda; z) = -\tfrac{1}{2} \lVert u(z) + \lambda a \rVert_2^2 + z^\top b - \lambda d.
$$

Đạo hàm theo \(\lambda\):

$$
\frac{d}{d\lambda} g(\lambda; z)
= - (u(z) + \lambda a)^\top a - d
= - u(z)^\top a - \lambda \lVert a \rVert_2^2 - d.
$$

Cho bằng 0:

$$
- u(z)^\top a - \lambda \lVert a \rVert_2^2 - d = 0
\quad \Rightarrow \quad
\lambda(z)^* = - \frac{u(z)^\top a + d}{\lVert a \rVert_2^2}.
$$

Với dữ liệu cụ thể của đề:

- \(A = \begin{bmatrix}1&0\\0&2\end{bmatrix} \Rightarrow A^\top = A\),
- \(c = (-2, 1)^\top\),
- \(a = (1, 1)^\top \Rightarrow \|a\|^2 = 2\),
- \(d = 1\),

ta có
$$
u(z) = c + A^\top z
= \begin{bmatrix} -2 \\ 1 \end{bmatrix}
+ \begin{bmatrix} 1 & 0 \\ 0 & 2 \end{bmatrix}
\begin{bmatrix} z_1 \\ z_2 \end{bmatrix}
= \begin{bmatrix} -2 + z_1 \\ 1 + 2 z_2 \end{bmatrix}.
$$

Khi đó

$$
u(z)^\top a = (-2 + z_1) + (1 + 2 z_2) = -1 + z_1 + 2 z_2.
$$

Thế vào công thức \(\lambda^*(z)\):

$$
\lambda^*(z)
= - \frac{(-1 + z_1 + 2 z_2) + 1}{2}
= - \frac{z_1 + 2 z_2}{2}.
$$

Vậy với mỗi \(z\) thỏa \(\|z\|\le1\), nghiệm đối ngẫu tối ưu theo \(\lambda\) là

$$
\boxed{\lambda^*(z) = - \dfrac{z_1 + 2 z_2}{2}}.
$$

---

### 3. Từ \(\lambda^*\), suy ra nghiệm primal tối ưu \(x^*\)

Nhớ rằng ở bước inf theo \(x\) ta đã có công thức:

$$
x(\lambda, z) = - \big( c + A^\top z + \lambda a \big).
$$

Thay dữ liệu cụ thể vào:

- \(c = (-2, 1)^\top\)
- \(A^\top z = (z_1, 2 z_2)^\top\)
- \(a = (1,1)^\top\)

ta được

$$
x(\lambda, z)
= - \begin{bmatrix}
-2 + z_1 + \lambda \\
1 + 2 z_2 + \lambda
\end{bmatrix}
= \begin{bmatrix}
2 - z_1 - \lambda \\
-1 - 2 z_2 - \lambda
\end{bmatrix}.
$$

Nếu ta dùng đúng \(\lambda = \lambda^*(z) = - \frac{z_1 + 2 z_2}{2}\), thì

\[
\begin{aligned}
x_1^*(z)
&= 2 - z_1 - \left(- \frac{z_1 + 2 z_2}{2}\right)
= 2 - z_1 + \frac{z_1 + 2 z_2}{2}
= 2 - \frac{z_1}{2} + z_2, \\
x_2^*(z)
&= -1 - 2 z_2 - \left(- \frac{z_1 + 2 z_2}{2}\right)
= -1 - 2 z_2 + \frac{z_1 + 2 z_2}{2}
= -1 + \frac{z_1}{2} - z_2.
\end{aligned}
\]

Ta có thể kiểm tra lại ràng buộc tuyến tính:

$$
x_1^*(z) + x_2^*(z)
= \Big(2 - \frac{z_1}{2} + z_2\Big) + \Big(-1 + \frac{z_1}{2} - z_2\Big) = 1,
$$

nên ràng buộc \(x_1 + x_2 = 1\) được thỏa **tự động** → đúng như cách ta xây đối ngẫu.

---

### 4. Kiểm tra primal = dual

- Giá trị primal tại \(x^*(z)\):
  $$
  p(z) = \tfrac{1}{2}\|x^*(z)\|^2 + c^\top x^*(z) + \|A x^*(z) + b\|_2.
  $$

- Giá trị dual tại \((\lambda^*(z), z)\):
  $$
  d(z) = - \tfrac{1}{2} \|c + A^\top z + \lambda^*(z) a\|_2^2 + z^\top b - \lambda^*(z) d.
  $$

Vì bài toán gốc lồi, ràng buộc tuyến tính và chuẩn 2 là hàm lồi đóng, điều kiện Slater thỏa (vẫn có điểm khả thi), nên ta có **đối ngẫu mạnh** và có thể kiểm tra lại rằng với \(z\) tối ưu (tức là nghiệm của bài toán dual), ta sẽ được

$$
p^* = d^*.
$$

(Trong bài nộp, bạn chỉ cần chỉ ra công thức này và giải thích: “do chuẩn 2 là hàm lồi đóng, ràng buộc là affine nên strong duality giữ” — là đủ.)

---

Nếu bạn muốn, mình có thể thêm **1 đoạn code Python ngắn** để:
1. duyệt lưới \(z\) trong hình tròn \(\|z\|\le1\),
2. tính \(\lambda^*(z)\), \(x^*(z)\), \(p(z)\), \(d(z)\),
3. chọn cặp nào cho chênh lệch \(|p(z)-d(z)|\) nhỏ nhất → đó là nghiệm gần đúng.


In [ ]:
import numpy as np

# ================================
# 1️⃣ DỮ LIỆU BÀI TOÁN
# ================================
A = np.array([[1, 0],
              [0, 2]])
b = np.array([-1, 1])
c = np.array([-2, 1])
a = np.array([1, 1])
d = 1


# ================================
# 2️⃣ HÀM TÍNH λ*(z) VÀ x*(z)
# ================================
def lambda_star(z, a=a, c=c, A=A, d=d):
    """
    Tính λ*(z) = -(u(z)^T a + d) / ||a||^2
    trong đó u(z) = c + A^T z
    """
    u = c + A.T @ z
    return - (u @ a + d) / (a @ a)


def x_star(z, lam, c=c, A=A, a=a):
    """
    Tính nghiệm primal x*(z) = - (c + A^T z + λ a)
    """
    return - (c + A.T @ z + lam * a)


# ================================
# 3️⃣ HÀM MỤC TIÊU PRIMAL & DUAL
# ================================
def primal_value(x):
    """
    p(x) = 0.5||x||^2 + c^T x + ||A x + b||
    """
    return 0.5 * np.dot(x, x) + np.dot(c, x) + np.linalg.norm(A @ x + b)


def dual_value(z, lam, A=A, b=b, c=c, a=a, d=d):
    """
    d(z, λ) = -0.5 ||c + A^T z + λ a||^2 + z^T b - λ d
    """
    return -0.5 * np.linalg.norm(c + A.T @ z + lam * a) ** 2 + np.dot(z, b) - lam * d


# ================================
# 4️⃣ DUYỆT LƯỚI z TRONG QUẢ CẦU ||z|| ≤ 1
# ================================
Z = []
vals = []

grid = np.linspace(-1, 1, 101)
for z1 in grid:
    for z2 in grid:
        if np.linalg.norm([z1, z2]) <= 1 + 1e-6:
            z = np.array([z1, z2])
            lam = lambda_star(z)
            x = x_star(z, lam)
            p = primal_value(x)
            dval = dual_value(z, lam)
            vals.append((z1, z2, lam, p, dval, abs(p - dval)))
            Z.append(z)

# ================================
# 5️⃣ CHỌN NGHIỆM CÓ |p - d| NHỎ NHẤT
# ================================
best = min(vals, key=lambda v: v[-1])
z_opt = np.array([best[0], best[1]])
lam_opt = best[2]
x_opt = x_star(z_opt, lam_opt)
p_opt, d_opt = best[3], best[4]

print("✅ Nghiệm đối ngẫu và primal tối ưu xấp xỉ:")
print(f"z* = {z_opt}")
print(f"λ* = {lam_opt:.4f}")
print(f"x* = {x_opt}")
print(f"p* = {p_opt:.6f}")
print(f"d* = {d_opt:.6f}")
print(f"|p* - d*| = {abs(p_opt - d_opt):.2e}")


# Phan newton

## **Câu 2 (2 điểm).**
Xét hàm số:

$$
f(x) = e^x - 3x^2 + 2x - 1.
$$

---

### **Yêu cầu:**

1. Viết công thức đạo hàm bậc nhất và bậc hai của \(f(x)\).  
2. Viết hàm thực hiện **thuật toán Newton 1 chiều tổng quát** để tìm nghiệm cực tiểu của hàm.  
3. Chạy thuật toán Newton từ nhiều điểm khởi tạo khác nhau \(x_0 = -2, 0, 2\).  
4. Vẽ đồ thị hàm \(f(x)\) trên đoạn \([-3, 3]\), **biểu diễn các điểm lặp trên đồ thị**.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# =====================================================
# 🎯 Yêu cầu 1: Định nghĩa hàm, đạo hàm bậc 1 và bậc 2
# =====================================================

def f_exp(x):
    """Hàm số f(x) = e^x - 3x^2 + 2x - 1"""
    return np.exp(x) - 3*x**2 + 2*x - 1

def df_exp(x):
    """Đạo hàm bậc 1: f'(x) = e^x - 6x + 2"""
    return np.exp(x) - 6*x + 2

def d2f_exp(x):
    """Đạo hàm bậc 2: f''(x) = e^x - 6"""
    return np.exp(x) - 6


# =====================================================
# 🎯 Yêu cầu 2: Viết hàm thực hiện thuật toán Newton 1D
# =====================================================

def newton_1d(f, df, d2f, x0, tol=1e-8, max_iter=50, use_backtracking=False):
    """
    Thuật toán Newton 1 chiều tổng quát để tìm cực trị.

    Tham số:
    --------
    f, df, d2f : callable
        Các hàm f(x), f'(x), f''(x)
    x0 : float
        Điểm khởi tạo
    tol : float
        Ngưỡng dừng theo |f'(x)| < tol
    max_iter : int
        Số vòng lặp tối đa
    use_backtracking : bool
        Nếu True -> dùng Armijo line search để điều chỉnh bước đi

    Trả về:
    --------
    x_star : float
        Nghiệm tối ưu tìm được
    history : list[float]
        Danh sách các điểm lặp để vẽ
    """
    x = float(x0)
    history = [x]

    for _ in range(max_iter):
        g = df(x)
        H = d2f(x)

        # Dừng nếu gradient nhỏ
        if abs(g) < tol:
            break

        # Tránh chia cho 0
        if abs(H) < 1e-12:
            H = np.sign(H) * 1e-12

        step = -g / H

        # Nếu dùng backtracking (tùy chọn)
        if use_backtracking:
            alpha = 1.0
            while f(x + alpha * step) > f(x) + 1e-4 * alpha * g * step:
                alpha *= 0.5
            step *= alpha

        x = x + step
        history.append(x)

    return x, history


# =====================================================
# 🎯 Yêu cầu 3 & 4: Chạy Newton và vẽ đồ thị các điểm lặp
# =====================================================

def run_and_plot_newton(f, df, d2f, inits, x_range=(-3, 3)):
    """
    Chạy thuật toán Newton từ nhiều điểm khởi tạo và vẽ đồ thị.

    Tham số:
    --------
    f, df, d2f : callable
        Các hàm f(x), f'(x), f''(x)
    inits : list[float]
        Danh sách các điểm khởi tạo x0
    x_range : tuple(float, float)
        Miền vẽ đồ thị (min, max)
    """
    xs = np.linspace(x_range[0], x_range[1], 400)
    ys = f(xs)

    plt.figure(figsize=(7, 4))
    plt.plot(xs, ys, label="f(x)", color='blue')
    colors = plt.cm.Set1(np.linspace(0, 1, len(inits)))

    print("=== KẾT QUẢ THUẬT TOÁN NEWTON ===")
    for i, x0 in enumerate(inits):
        x_star, hist = newton_1d(f, df, d2f, x0)
        print(f"Khởi tạo x0 = {x0:>5}:  x* ≈ {x_star:.6f},  số bước = {len(hist)-1}")

        # Vẽ các điểm lặp
        plt.scatter(hist, [f(h) for h in hist], s=40, color=colors[i], label=f"x0 = {x0}")
        for j in range(len(hist)-1):
            plt.plot([hist[j], hist[j+1]],
                     [f(hist[j]), f(hist[j+1])],
                     color=colors[i], linewidth=1)

    plt.title("Thuật toán Newton 1D cho f(x) = e^x - 3x² + 2x - 1")
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.legend()
    plt.grid(True)
    plt.show()


# =====================================================
# 🎯 GỌI DEMO TOÀN BỘ QUY TRÌNH
# =====================================================

if __name__ == "__main__":
    # Các điểm khởi tạo
    initial_points = [-2, 0, 2]

    # Chạy toàn bộ yêu cầu 3 + 4
    run_and_plot_newton(f_exp, df_exp, d2f_exp, initial_points)


## **Câu 2 (3 điểm) — Newton 2 biến (nâng cao)**

Xét hàm số hai biến:
\[
f(x, y) = e^{x + y} + (x - 1)^2 + (y + 2)^2.
\]

---

### **Yêu cầu:**

1. **(Lý thuyết)**  
   a. Viết biểu thức gradient \(\nabla f(x, y)\).  
   b. Viết ma trận Hessian \(H(x, y)\).

2. **(Thuật toán Newton hai biến)**  
   Viết hàm tổng quát thực hiện **thuật toán Newton** cho hàm \(f(x, y)\):
   \[
   (x_{k+1}, y_{k+1}) = (x_k, y_k) - H(x_k, y_k)^{-1} \, \nabla f(x_k, y_k).
   \]

3. **(Thực nghiệm)**  
   Chạy thuật toán Newton với các điểm khởi tạo khác nhau:
   \[
   (x_0, y_0) \in \{\,(-3, -3), \; (0, 0), \; (2, 2)\,\}.
   \]
   - In ra điểm hội tụ \((x^*, y^*)\) và số vòng lặp.  
   - Nhận xét sự hội tụ nhanh/chậm giữa các điểm khởi tạo.

4. **(Trực quan hóa)**  
   - Vẽ đồ thị **đường đồng mức (contour)** của \(f(x, y)\) trên một miền thích hợp (ví dụ \([-4,4] \times [-4,4]\)).  
   - Biểu diễn **các điểm lặp Newton** trên đồ thị.  
   - Đánh dấu **điểm cực tiểu** bằng ký hiệu đặc biệt (màu/marker khác).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from autograd import grad, hessian
from autograd import numpy as anp  # autograd dùng phiên bản numpy riêng

# ==========================================================
# 🎯 1️⃣ Định nghĩa hàm f(x, y)
# ==========================================================
def f_xy_autograd(xy):
    """Hàm f(x, y) = e^(x+y) + (x-1)^2 + (y+2)^2"""
    x, y = xy
    return anp.exp(x + y) + (x - 1)**2 + (y + 2)**2

# Tự động tạo gradient & Hessian nhờ autograd
grad_f = grad(f_xy_autograd)
hessian_f = hessian(f_xy_autograd)


# ==========================================================
# 🎯 2️⃣ Newton 2D (dùng thư viện)
# ==========================================================
def newton_2d_auto(f, grad_f, hess_f, xy0, tol=1e-8, max_iter=50):
    """
    Newton 2D tổng quát sử dụng autograd.
    f, grad_f, hess_f : callable
        Hàm, gradient, và Hessian sinh tự động
    xy0 : (x0, y0)
        Điểm khởi tạo
    """
    xy = np.array(xy0, dtype=float)
    history = [xy.copy()]

    for _ in range(max_iter):
        g = np.array(grad_f(xy))
        H = np.array(hess_f(xy))

        if np.linalg.norm(g) < tol:
            break

        try:
            step = np.linalg.solve(H, -g)
        except np.linalg.LinAlgError:
            step = -np.linalg.pinv(H) @ g  # fallback nếu H suy biến

        xy += step
        history.append(xy.copy())

    return xy, np.array(history)


# ==========================================================
# 🎯 3️⃣ Trực quan hóa đường đi Newton
# ==========================================================
def plot_newton_paths(f, paths, x_range=(-4, 4), y_range=(-4, 4)):
    x = np.linspace(*x_range, 200)
    y = np.linspace(*y_range, 200)
    X, Y = np.meshgrid(x, y)
    Z = np.vectorize(lambda a, b: f(np.array([a, b])))(X, Y)

    plt.figure(figsize=(8, 6))
    CS = plt.contour(X, Y, Z, levels=30, cmap="viridis")
    plt.clabel(CS, inline=True, fontsize=8)

    colors = ["red", "orange", "green", "purple"]
    for i, hist in enumerate(paths):
        plt.plot(hist[:, 0], hist[:, 1], "-o", color=colors[i],
                 label=f"Path {i+1} (x0, y0)={hist[0]}")
        plt.scatter(hist[-1, 0], hist[-1, 1], s=80, c=colors[i], marker="*", edgecolors='k')

    plt.title("Đường đi hội tụ Newton 2 biến với autograd")
    plt.xlabel("x"); plt.ylabel("y")
    plt.legend(); plt.grid(True)
    plt.show()


# ==========================================================
# 🎯 4️⃣ DEMO TOÀN BỘ
# ==========================================================
if __name__ == "__main__":
    starts = [(-3, -3), (0, 0), (2, 2)]
    histories = []

    for s in starts:
        xy_star, hist = newton_2d_auto(f_xy_autograd, grad_f, hessian_f, s)
        histories.append(hist)
        print(f"Khởi tạo {s} → Hội tụ về {xy_star}, f* = {f_xy_autograd(xy_star):.6f}")

    plot_newton_paths(f_xy_autograd, histories)


## **Câu 2 (3 điểm) — Newton 2 biến (nâng cao + Backtracking Line Search)**

Xét hàm số hai biến:

**$f(x, y) = e^{x + y} + (x - 1)^2 + (y + 2)^2$**

---

### **Yêu cầu:**

1. **(Lý thuyết)**  
   a. Viết biểu thức gradient $\nabla f(x, y)$.  
   b. Viết ma trận Hessian $H(x, y)$.  
   c. Phát biểu **quy tắc dừng Armijo** trong backtracking line search.

---

2. **(Thuật toán Newton hai biến có Backtracking)**  
   Viết hàm tổng quát thực hiện **thuật toán Newton 2 biến có line search** theo quy tắc Armijo:

   $$
   (x_{k+1}, y_{k+1}) = (x_k, y_k) + \alpha_k d_k
   $$

   trong đó:

   $$
   d_k = -H(x_k, y_k)^{-1} \nabla f(x_k, y_k)
   $$

   và $\alpha_k$ được tìm bằng thuật toán **Backtracking Line Search**:

   $$
   \alpha_k = \max\{\,\beta^m : f(x_k + \beta^m d_k) \le f(x_k) + c \beta^m \nabla f(x_k)^T d_k \,\}
   $$

   với $c \in (0, 1)$, $0 < \beta < 1$.

---

3. **(Thực nghiệm)**  
   Chạy thuật toán Newton–Backtracking với các điểm khởi tạo:

   **$(x_0, y_0) \in \{(-3, -3), (0, 0), (2, 2)\}$**

   - In ra:
     - Điểm hội tụ $(x^*, y^*)$,  
     - Giá trị $f(x^*, y^*)$,  
     - Số vòng lặp,  
     - Số lần giảm bước mỗi vòng (backtrack count).  

   - So sánh tốc độ hội tụ với **Newton không backtracking**.

---

4. **(Trực quan hóa)**  
   - Vẽ đồ thị **đường đồng mức (contour)** của $f(x, y)$.  
   - Biểu diễn **các đường đi Newton có/không backtracking** trên cùng biểu đồ.  
   - Đánh dấu **điểm cực tiểu** bằng ký hiệu đặc biệt (★).  
   - Ghi chú rõ ràng từng đường đi và tốc độ hội tụ.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from autograd import grad, hessian
from autograd import numpy as anp

# ==========================================================
# 🎯 1️⃣ Định nghĩa hàm, gradient và Hessian (tự động)
# ==========================================================
def f_xy(v):
    """
    Hàm mục tiêu: f(x, y) = e^(x + y) + (x - 1)^2 + (y + 2)^2
    v : mảng 1D [x, y]
    """
    x, y = v
    return anp.exp(x + y) + (x - 1)**2 + (y + 2)**2

# Tự động sinh gradient và Hessian bằng autograd
grad_f = grad(f_xy)
hessian_f = hessian(f_xy)


# ==========================================================
# 🎯 2️⃣ Newton 2 biến có Backtracking Line Search (Armijo)
# ==========================================================
def newton_backtracking(f, grad_f, hess_f, x0, tol=1e-8, max_iter=50, c=1e-4, beta=0.5):
    """
    Thuật toán Newton có backtracking line search (Armijo)

    Tham số
    --------
    f, grad_f, hess_f : callable
        Hàm mục tiêu, gradient và Hessian
    x0 : ndarray (2,)
        Điểm khởi tạo [x0, y0]
    tol : float
        Ngưỡng dừng theo chuẩn của gradient
    max_iter : int
        Số vòng lặp tối đa
    c, beta : float
        Hệ số Armijo (mặc định: c=1e-4, beta=0.5)
    """
    x = np.array(x0, dtype=float)
    history = [x.copy()]

    for k in range(max_iter):
        g = np.array(grad_f(x))
        H = np.array(hess_f(x))

        # Dừng khi gradient đủ nhỏ
        if np.linalg.norm(g) < tol:
            break

        try:
            d = -np.linalg.solve(H, g)
        except np.linalg.LinAlgError:
            d = -np.linalg.pinv(H) @ g  # fallback nếu Hessian suy biến

        alpha = 1.0
        f0 = f(x)
        back_count = 0

        # Quy tắc Armijo
        while f(x + alpha * d) > f0 + c * alpha * np.dot(g, d):
            alpha *= beta
            back_count += 1
            if alpha < 1e-8:
                break

        x = x + alpha * d
        history.append(x.copy())

        print(f"Iter {k+1:02d}: f={f(x):.6f}, |∇f|={np.linalg.norm(g):.2e}, α={alpha:.3f}, back={back_count}")

    return x, np.array(history)


# ==========================================================
# 🎯 3️⃣ Newton không Backtracking (để so sánh)
# ==========================================================
def newton_plain(f, grad_f, hess_f, x0, tol=1e-8, max_iter=50):
    """Thuật toán Newton thuần (không line search)."""
    x = np.array(x0, dtype=float)
    history = [x.copy()]

    for _ in range(max_iter):
        g = np.array(grad_f(x))
        H = np.array(hess_f(x))

        if np.linalg.norm(g) < tol:
            break

        try:
            d = -np.linalg.solve(H, g)
        except np.linalg.LinAlgError:
            d = -np.linalg.pinv(H) @ g

        x = x + d
        history.append(x.copy())

    return x, np.array(history)


# ==========================================================
# 🎯 4️⃣ Hàm trực quan hóa (contour + đường đi)
# ==========================================================
def plot_paths(f, paths, labels, colors, x_range=(-4, 4), y_range=(-4, 4)):
    """
    Vẽ contour và đường đi của các thuật toán Newton.
    """
    x = np.linspace(*x_range, 300)
    y = np.linspace(*y_range, 300)
    X, Y = np.meshgrid(x, y)
    Z = np.vectorize(lambda a, b: f(np.array([a, b])))(X, Y)

    plt.figure(figsize=(8, 6))
    plt.contour(X, Y, Z, levels=40, cmap="viridis")

    for hist, lab, col in zip(paths, labels, colors):
        plt.plot(hist[:, 0], hist[:, 1], "-o", color=col, label=lab)
        plt.scatter(hist[-1, 0], hist[-1, 1], s=100, marker="*", c=col, edgecolors="k")

    plt.xlabel("x")
    plt.ylabel("y")
    plt.title("Đường hội tụ Newton 2 biến (có & không backtracking)")
    plt.legend()
    plt.grid(True)
    plt.show()


# ==========================================================
# 🎯 5️⃣ Demo toàn bộ
# ==========================================================
if __name__ == "__main__":
    starts = [(-3, -3), (0, 0), (2, 2)]

    all_paths_bt, all_paths_plain = [], []

    print("=== Newton có Backtracking ===")
    for s in starts:
        x_star, hist = newton_backtracking(f_xy, grad_f, hessian_f, s)
        all_paths_bt.append(hist)
        print(f"Khởi tạo {s} → Hội tụ: {x_star}, f* = {f_xy(x_star):.6f}\n")

    print("=== Newton không Backtracking ===")
    for s in starts:
        x_star, hist = newton_plain(f_xy, grad_f, hessian_f, s)
        all_paths_plain.append(hist)
        print(f"Khởi tạo {s} → Hội tụ: {x_star}, f* = {f_xy(x_star):.6f}\n")

    # Vẽ đồ thị so sánh
    plot_paths(f_xy,
               paths=all_paths_bt + all_paths_plain,
               labels=[f"BT từ {s}" for s in starts] + [f"Plain từ {s}" for s in starts],
               colors=["red", "orange", "green", "blue", "purple", "gray"])
